<center>
<h2>CALDEN CONSULTORIA - Leitura Dados FME e Mapas
</h2>
<h3>Projeto: Invetario de Campo</h3>
</center>
</left>
<h3>Cliente: Light</h3>
<h4>Notebook 01: Leitura
    

# PAth

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
from os import listdir
import zipfile
import time
from datetime import datetime
from shapely.ops import unary_union
# import openpyxl
import glob
import pickle
# import pygeos
from shapely.geometry import Point, LineString
import folium
import pyproj
# import cv2
import matplotlib.pyplot as plt
from folium import plugins
import xlsxwriter



#import folium
#from folium import plugins
#from folium import Map, FeatureGroup, Marker, LayerControl, Popup
#from folium.features import DivIcon
import branca.colormap as cm

pd.options.display.max_rows = 10

#import atrb_mcpse

In [2]:
path_ext = 'C:/Users/fzaqu/Light/Inventário 1º Onda 2023_Light - General/01 - Frentes do Inventário/01 - Inventário de Postes/01 - Extracoes FME/16-02-24/'

In [137]:
extracao = "01_13.12.2023_Extracao"

In [7]:
path_escopo = "D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/0. Escopo/"

In [8]:
escopo = "ALIM VALIDADOS - 12.12.2023.xlsx"

In [3]:
path_resultados = "C:/GIT/2023_Light_Dash/BASE_DADOS/"

# Leitura escopo

In [41]:
df_escopo = pd.read_excel(path_escopo + escopo, sheet_name= "DADOS" )
df_escopo.columns = df_escopo.iloc[0]

# Descartar as primeiras duas linhas
df_escopo = df_escopo.drop([0])

# Resetar o índice
df_escopo = df_escopo.reset_index(drop=True)


In [42]:
len(df_escopo)

339

In [43]:
df_escopo

,ITEM,REGIONAL,ALIMENTADOR,SUBESTAÇÃO,POSTE,TRAFO,RELIGAD.,SECCION.,KM_MT,KM_BT,KM_RAMAIS,QTDE RAMAIS,CLIENTES BT
0,273,OESTE,LDS 61297,SETD GARDENIA,12,4,0,0,3.264561,0,0.062183,14,NaN
1,340,OESTE,LDA BANDER,SETD CURICICA,12,0,0,0,2.694859,0,0,0,NaN
2,341,OESTE,LDS 61299,SETD GARDENIA,12,0,0,0,4.924825,0,0,0,NaN
3,339,OESTE,LDA NOBET,SETD JABOATAO,13,0,0,0,2.008086,0,0,0,NaN
4,275,OESTE,LDS 61139,SETD GARDENIA,16,4,0,0,3.172677,0.022557,0.077588,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,119,BAIXADA,LDA TABOLEIRO,SESD TUIUTI,1492,138,4,5,54.869917,43.22154,24.541077,1614,NaN
335,284,BAIXADA,LDA HORTENCIA/CENTAUREA,SETD COMENDADOR SOARES,1517,76,3,2,0,0,0,0,NaN
336,96,OESTE,LDA VILAMIL,SETD PADRE MIGUEL,1553,135,5,5,32.474668,34.030487,26.744972,1986,NaN
337,2,CENTRO SUL,LDA RIOMORTO,SETD RECREIO,1645,152,4,10,31.790495,56.096736,67.980405,6316,NaN


# Leitura trechos

In [44]:
gdf_trecho_mt = gpd.read_file(path_ext +extracao+ "/TRECHO_MT.shp")
gdf_trecho_mt['geometry'] = gdf_trecho_mt['geometry'].to_crs(4326)

In [45]:
gdf_trecho_mt = gdf_trecho_mt.set_geometry('geometry')

In [46]:
gdf_trecho_mt.columns

Index(['CODIGO', 'COMPRIMENT', 'COMPRIME00', 'FASES', 'TIPO_INSTA',
       'TENSAO_OPE', 'PROPRIETAR', 'MAX_LINHA', 'MAX_MODELO', 'ITEM_ESTOQ',
       'BITOLA_FAS', 'MATERIAL_F', 'ISOLAMENTO', 'ITEM_EST00', 'BITOLA_NEU',
       'MATERIAL_N', 'ISOLAMEN00', 'TIPO_COBER', 'ENCORDOAME', 'A1', 'A2',
       'A3', 'A4', 'A5', 'A6', 'A1_NEUTRO', 'A2_NEUTRO', 'A3_NEUTRO',
       'A4_NEUTRO', 'A5_NEUTRO', 'A6_NEUTRO', 'CODIGO_LIN', 'LINHA',
       'MAX_ALIMEN', 'NOME', 'ALIMENTADO', 'LONG', 'LAT', 'Q__z', 'geometry'],
      dtype='object')

In [47]:
gdf_trecho_mt.head()

,CODIGO,COMPRIMENT,COMPRIME00,FASES,TIPO_INSTA,TENSAO_OPE,PROPRIETAR,MAX_LINHA,MAX_MODELO,ITEM_ESTOQ,...,A6_NEUTRO,CODIGO_LIN,LINHA,MAX_ALIMEN,NOME,ALIMENTADO,LONG,LAT,Q__z,geometry
0,617676.0,30.2655,17.4183,ABC,Aéreo,NaN,Light,615303,105925008,323201,...,00,615303.0,LDA CAMELIA,613501,LDA CAMELIA,LDA CAMELIA,-43.371689,-22.876506,NaN,"LINESTRING (-43.37169 -22.87651, -43.37161 -22..."
1,617700.0,26.1069,21.1069,ABC,Aéreo,13.8,Light,615303,105925005,323002,...,00,615303.0,LDA CAMELIA,613501,LDA CAMELIA,LDA CAMELIA,-43.371629,-22.876328,NaN,"LINESTRING (-43.37163 -22.87633, -43.37172 -22..."
2,653264.0,56.9716,56.9716,ABC,Aéreo,NaN,Light,615100,105925008,323201,...,00,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372710,-22.886731,NaN,"LINESTRING (-43.37271 -22.88673, -43.37310 -22..."
3,653360.0,39.1192,39.1192,ABC,Aéreo,NaN,Light,615100,105925008,323201,...,00,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.371362,-22.888049,NaN,"LINESTRING (-43.37136 -22.88805, -43.37162 -22..."
4,653390.0,63.6026,63.6026,ABC,Aéreo,NaN,Light,615100,105925008,323201,...,00,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372033,-22.887383,NaN,"LINESTRING (-43.37203 -22.88738, -43.37247 -22..."


In [48]:
gdf_trecho_mt['LINHA'].value_counts()

LINHA
LDA RODU           1413
LDA BELAVI         1406
LDA TABOLEIRO      1401
LDA VILAMIL        1266
LDA JEQUITIBA      1166
                   ... 
LDA PROJAC           32
LDA TOPSHOPPING      30
LDS 24794             9
LRA R SEVERIANO       9
LDS 29436             1
Name: count, Length: 339, dtype: int64

In [51]:
gdf_trecho_mt['CODIGO'] = gdf_trecho_mt['CODIGO'].astype(int)
gdf_trecho_mt['COMPRIMENT'] = gdf_trecho_mt['COMPRIMENT'].astype(float)

In [57]:
gdf_trecho_mt.head()

,CODIGO,COMPRIMENT,COMPRIME00,FASES,TIPO_INSTA,TENSAO_OPE,PROPRIETAR,MAX_LINHA,MAX_MODELO,ITEM_ESTOQ,...,A6_NEUTRO,CODIGO_LIN,LINHA,MAX_ALIMEN,NOME,ALIMENTADO,LONG,LAT,Q__z,geometry
0,617676,30.2655,17.4183,ABC,Aéreo,NaN,Light,615303,105925008,323201,...,00,615303.0,LDA CAMELIA,613501,LDA CAMELIA,LDA CAMELIA,-43.371689,-22.876506,NaN,"LINESTRING (-43.37169 -22.87651, -43.37161 -22..."
1,617700,26.1069,21.1069,ABC,Aéreo,13.8,Light,615303,105925005,323002,...,00,615303.0,LDA CAMELIA,613501,LDA CAMELIA,LDA CAMELIA,-43.371629,-22.876328,NaN,"LINESTRING (-43.37163 -22.87633, -43.37172 -22..."
2,653264,56.9716,56.9716,ABC,Aéreo,NaN,Light,615100,105925008,323201,...,00,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372710,-22.886731,NaN,"LINESTRING (-43.37271 -22.88673, -43.37310 -22..."
3,653360,39.1192,39.1192,ABC,Aéreo,NaN,Light,615100,105925008,323201,...,00,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.371362,-22.888049,NaN,"LINESTRING (-43.37136 -22.88805, -43.37162 -22..."
4,653390,63.6026,63.6026,ABC,Aéreo,NaN,Light,615100,105925008,323201,...,00,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372033,-22.887383,NaN,"LINESTRING (-43.37203 -22.88738, -43.37247 -22..."


In [81]:
extracao

'01_13.12.2023_Extracao/'

# Saida trechos mapas

In [55]:
gdf_trecho_mt[[ 'CODIGO','COMPRIMENT', 'LINHA','geometry' ]].merge(df_escopo[["ALIMENTADOR", "REGIONAL"]], right_on = "ALIMENTADOR", left_on = 'LINHA').to_pickle(path_resultados + extracao+'_Trechos_MT_mapas.pkl')

In [165]:
saida_trecho_mapa = gdf_trecho_mt[[ 'CODIGO','COMPRIMENT', 'LINHA','geometry' ]].merge(df_escopo[["ALIMENTADOR", "REGIONAL"]], right_on = "ALIMENTADOR", left_on = 'LINHA')

In [172]:
saida_trecho_mapa

,CODIGO,COMPRIMENT,LINHA,geometry,ALIMENTADOR,REGIONAL
0,617676,30.2655,LDA CAMELIA,"LINESTRING (-43.37169 -22.87651, -43.37161 -22...",LDA CAMELIA,OESTE
1,617700,26.1069,LDA CAMELIA,"LINESTRING (-43.37163 -22.87633, -43.37172 -22...",LDA CAMELIA,OESTE
2,672986,13.7473,LDA CAMELIA,"LINESTRING (-43.37116 -22.87930, -43.37113 -22...",LDA CAMELIA,OESTE
3,673094,42.4852,LDA CAMELIA,"LINESTRING (-43.37085 -22.87798, -43.37076 -22...",LDA CAMELIA,OESTE
4,673100,29.9140,LDA CAMELIA,"LINESTRING (-43.37076 -22.87760, -43.37069 -22...",LDA CAMELIA,OESTE
...,...,...,...,...,...,...
111845,572320172,24.3267,LDA ALLENDE,"LINESTRING (-43.41894 -22.98995, -43.41874 -22...",LDA ALLENDE,CENTRO SUL
111846,572320182,39.4015,LDA ALLENDE,"LINESTRING (-43.41874 -22.98982, -43.41838 -22...",LDA ALLENDE,CENTRO SUL
111847,572320192,8.6800,LDA ALLENDE,"LINESTRING (-43.41838 -22.98970, -43.41832 -22...",LDA ALLENDE,CENTRO SUL
111848,572320202,67.1464,LDA ALLENDE,"LINESTRING (-43.41832 -22.98965, -43.41898 -22...",LDA ALLENDE,CENTRO SUL


In [173]:
saida_trecho_mapa.columns = ['CODIGO', 'COMPRIMENTO', 'ALIMENTADOR', 'geometry', 'ali', 'REGIONAL']

In [175]:
saida_trecho_mapa[['CODIGO', 'COMPRIMENTO', 'ALIMENTADOR', 'geometry', 'REGIONAL']].to_pickle(path_resultados + extracao +  '_Trechos_alimentadores.pkl')

# Status BI

In [5]:
path_input = 'D:/CALDEN/[2022-23] LIGHT - Sustentabilidade Contrato Concessão - Inventário 1ª Onda/BI/10.11.2023_Input_Status_Inventario_Light_prot_v01.xlsm'

In [14]:
df_log = pd.read_excel(path_input, sheet_name=  'Log', skiprows = 6)
df_etapas = pd.read_excel(path_input, sheet_name=  'Etapas')

In [18]:
df_etapas

,Etapa,Status Alimentador,Status,Responsável,Macro-Etapas,Macro-Status,T_etapas,T_macro_etapas,T_etapas_acum,T_macro_acum
0,0,não iniciado,00. não iniciado,-,0,-,0,0,0,0
1,1,em planejamento,01. em planejamento,Light,0,Em planejamento,3,5,0,0
2,2,em exportação,02. em exportação,Codex,0,Em planejamento,1,5,3,0
3,3,em auditoria (fase 0),03. em auditoria (fase 0),Light,0,Em planejamento,1,5,4,0
4,4,em levantamento de campo,04. em levantamento de campo,Delos,1,Em campo,15,45,5,5
...,...,...,...,...,...,...,...,...,...,...
12,12,em conciliação,12. em conciliação,Light,3,Em conc físico-contábil,10,20,70,70
13,13,em auditoria qualidade (fase 3),13. em auditoria qualidade (fase 3),Calden,3,Em conc físico-contábil,5,20,80,70
14,14,em análise contábil,14. em análise contábil,Light,3,Em conc físico-contábil,5,20,85,70
15,15,concluído,15. concluído,Light,4,Concluído,10,10,100,100


In [15]:
df_log

,ID,Alimentador,Status,Responsável,Usuário,Data de Atualização
0,0,LDA ABAETE,00. não iniciado,-,-,-
1,1,LDA ABATOS,00. não iniciado,-,-,-
2,2,LDA ABELARDO,00. não iniciado,-,-,-
3,3,LDA ACHE,00. não iniciado,-,-,-
4,4,LDA ADUTORA,00. não iniciado,-,-,-
...,...,...,...,...,...,...
5044,5044,NaN,NaN,NaN,NaN,NaN
5045,5045,NaN,NaN,NaN,NaN,NaN
5046,5046,NaN,NaN,NaN,NaN,NaN
5047,5047,NaN,NaN,NaN,NaN,NaN


In [16]:
df_log['ID'] = pd.to_numeric(df_log['ID'], errors='coerce')

# Encontrar a linha com o ID máximo para cada alimentador
linha_max_id_por_alimentador = df_log.loc[df_log.groupby('Alimentador')['ID'].idxmax()]

In [13]:
linha_max_id_por_alimentador

,ID,Alimentador,Status,Responsável,Usuário,Data de Atualização
1020,1020,LDA ABAETE,01. em planejamento,User,Tiago,2024-01-22 00:00:00
1021,1021,LDA ABATOS,01. em planejamento,User,Tiago,2024-01-22 00:00:00
1022,1022,LDA ABELARDO,01. em planejamento,User,Tiago,2024-01-22 00:00:00
1032,1032,LDA ACHE,07. em liberação para importação GIS,User,Tiago,2024-01-22 00:00:00
1024,1024,LDA ADUTORA,01. em planejamento,User,Tiago,2024-01-22 00:00:00
...,...,...,...,...,...,...
1033,1033,LSA RODUTRA,16. inventário paralisado,User,Tiago,2024-01-22 00:00:00
1034,1034,LSA SABIA,06. em revisão campo,User,Tiago,2024-01-22 00:00:00
1015,1015,LSA SAMPAIO,01. em planejamento,User,Tiago,2024-01-19 00:00:00
1016,1016,LSA SHALON,01. em planejamento,User,Tiago,2024-01-19 00:00:00


In [21]:
latest = linha_max_id_por_alimentador[['Alimentador', 'Status']].merge(df_etapas[[ 'Macro-Status', 'Status']], on = [ 'Status'], how = 'left' )

In [27]:
latest[['Alimentador', 'Macro-Status']]

,Alimentador,Macro-Status
0,LDA ABAETE,Em planejamento
1,LDA ABATOS,Em planejamento
2,LDA ABELARDO,Em planejamento
3,LDA ACHE,Em postagem
4,LDA ADUTORA,Em planejamento
...,...,...
334,LSA RODUTRA,Inventário Paralisado
335,LSA SABIA,Em campo
336,LSA SAMPAIO,Em planejamento
337,LSA SHALON,Em planejamento


In [26]:
latest[['Alimentador', 'Macro-Status']].to_pickle(path_resultados + 'Status_alimentador.pkl')

# Saida trechos BI

In [74]:
# Agrupar por 'Categoria' e calcular a soma e a contagem
BI_trecho = gdf_trecho_mt.groupby('LINHA').agg({'COMPRIMENT': ['sum']}).reset_index()

# Renomear as colunas
BI_trecho.columns = ['LINHA', 'REDE_MT_KM']

BI_trecho

,LINHA,REDE_MT_KM
0,LDA ABAETE,4801.0447
1,LDA ABATOS,17277.1451
2,LDA ABELARDO,4193.9757
3,LDA ACHE,5965.3535
4,LDA ADUTORA,59927.2544
...,...,...
334,LSA RODUTRA,34739.5379
335,LSA SABIA,9113.7148
336,LSA SAMPAIO,2833.5922
337,LSA SHALON,22343.2989


# TRAFO

In [5]:
gdf_TRAFO = gpd.read_file(path_ext + "/TRANSFORMADOR.shp")
gdf_TRAFO['geometry'] = gdf_TRAFO['geometry'].to_crs(4326)

In [6]:
gdf_TRAFO = gdf_TRAFO.set_geometry('geometry')

In [7]:
gdf_TRAFO.to_pickle(path_resultados + 'trafo.pkl')

In [8]:
gdf_TRAFO.columns

Index(['CODIGO', 'MATRICULA', 'CAPAC_CHAV', 'TIPO_INSTA', 'EXCLUSIVO_',
       'FASES', 'TIPO_TRANS', 'TIPO_LIGAC', 'TIPO_ATERR', 'POSTO',
       'TIPO_MONTA', 'CAPAC_ELO', 'AUTOPROTEG', 'IND_REDE_M', 'IND_PARAL',
       'POTENCIA_I', 'BLINDADO', 'MAX_CIRCUI', 'MAX_LINHA', 'MAX_EQTRAN',
       'LOTE', 'MEDICAO_BA', 'MATRICULA_', 'LP', 'CLASSE', 'FABRICANTE',
       'DATA_FABR', 'SERIE', 'QTDE_FASES', 'POTENCIA_N', 'TENSAO_PRI',
       'TENSAO_SEC', 'MEIO_ISOLA', 'TIPO_AUTO_', 'MAX_TRANSF', 'CODIGO_LIN',
       'LINHA', 'MAX_ALIMEN', 'ITEM', 'REGIONAL', 'ALIMENTADO', 'SUBESTAÇ',
       'POSTE', 'TRAFO', 'RELIGAD_', 'SECCION_', 'KM_MT', 'KM_BT', 'KM_RAMAIS',
       'QTDE RAMAI', 'CLIENTES B', 'LONG', 'LAT', 'Q__z', 'geometry'],
      dtype='object')

In [68]:
gdf_TRAFO.head()

,CODIGO,MATRICULA,CAPAC_CHAV,TIPO_INSTA,EXCLUSIVO_,FASES,TIPO_TRANS,TIPO_LIGAC,TIPO_ATERR,POSTO,...,MAX_TRANSF,CODIGO_LIN,LINHA,MAX_ALIMEN,NOME,ALIMENTADO,LONG,LAT,Q__z,geometry
0,748478984.0,ZNA437651,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,748478984,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.699292,-22.926614,NaN,POINT (-43.69929 -22.92661)
1,19746023.0,ZNA434125,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19746023,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.691641,-22.922198,NaN,POINT (-43.69164 -22.92220)
2,19719449.0,ZNA44040,100A,Aéreo,Não,ABC,Banco trifásico paralelo,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19719449,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.701437,-22.925699,NaN,POINT (-43.70144 -22.92570)
3,19719449.0,ZNA44040,100A,Aéreo,Não,ABC,Banco trifásico paralelo,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19719449,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.701437,-22.925699,NaN,POINT (-43.70144 -22.92570)
4,620617.0,ZNA28024,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,620617,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372470,-22.886976,NaN,POINT (-43.37247 -22.88698)


In [69]:
gdf_TRAFO['LINHA'].value_counts()

LINHA
LDA JEQUITIBA      175
LDA SANTAREM       171
LDA BELAVI         167
LDA GADOBAN        147
LDA RODU           147
                  ... 
LDA RIO 2            3
LDA RESERVA-ZIN      2
LDA OURO             2
LDA SALVALENDE       1
LDA GIBRAN           1
Name: count, Length: 287, dtype: int64

In [70]:
gdf_TRAFO['CODIGO'] = gdf_trecho_mt['CODIGO'].astype(int)
#gdf_trecho_mt['COMPRIMENT'] = gdf_trecho_mt['COMPRIMENT'].astype(float)

In [32]:
TRAFO = gdf_TRAFO[gdf_TRAFO['CODIGO'] == '157892']

In [33]:
TRAFO

,CODIGO,MATRICULA,CAPAC_CHAV,TIPO_INSTA,EXCLUSIVO_,FASES,TIPO_TRANS,TIPO_LIGAC,TIPO_ATERR,POSTO,...,SECCION_,KM_MT,KM_BT,KM_RAMAIS,QTDE RAMAI,CLIENTES B,LONG,LAT,Q__z,geometry


In [75]:
# Agrupar por 'Categoria' e calcular a soma e a contagem
BI_trafo= gdf_TRAFO.groupby('LINHA').agg({'CODIGO': ['count']}).reset_index()

# Renomear as colunas
BI_trafo.columns = ['LINHA', 'qtd_trafo']

BI_trafo

,LINHA,qtd_trafo
0,LDA ABAETE,29
1,LDA ABATOS,50
2,LDA ABELARDO,23
3,LDA ACHE,45
4,LDA ADUTORA,114
...,...,...
282,LRA PEIXOTO,8
283,LRA R CARLEME,23
284,LRA R PAÇUCAR,7
285,LRA R POLIXENA,7


# SUBESTAÇÃO

In [14]:
#Pasta subestação BDGD
path_bdgd = "C:/GITEA/2022_23_LIGHT_Sust_Concessao/4. Execução/2. BDGD/1. Dados/LIGHT_SHP_01092023/LIGHT_SHP_01092023"

In [15]:
gdf_SE = gpd.read_file(path_bdgd + "/SUB.shp")
gdf_SE['geometry'] = gdf_SE['geometry'].to_crs(4326)

In [16]:
gdf_SE = gdf_SE.set_geometry('geometry')

In [20]:
gdf_SE.to_pickle(path_resultados + 'SE.pkl')

# Ramal

In [82]:
gdf_ramal = gpd.read_file(path_ext +extracao+ "/RAMAL_SERVIÇO.shp")
gdf_ramal['geometry'] = gdf_ramal['geometry'].to_crs(4326)

In [83]:
gdf_ramal = gdf_ramal.set_geometry('geometry')

In [84]:
gdf_ramal.columns

Index(['CODIGO', 'COMPRIMENT', 'COMPRIME00', 'FASES', 'TIPO_INSTA',
       'RAMAL_SERV', 'PROPRIETAR', 'MAX_CIRCUI', 'MAX_MODELO', 'NOME',
       'MAX_TRANSF', 'MATRICULA', 'MAX_LINHA', 'ITEM_ESTOQ', 'BITOLA_FAS',
       'MATERIAL_F', 'ISOLAMENTO', 'ITEM_EST00', 'BITOLA_NEU', 'MATERIAL_N',
       'ISOLAMEN00', 'TIPO_COBER', 'ENCORDOAME', 'A1', 'A2', 'A3', 'A4', 'A5',
       'A6', 'A1_NEUTRO', 'A2_NEUTRO', 'A3_NEUTRO', 'A4_NEUTRO', 'A5_NEUTRO',
       'A6_NEUTRO', 'CODIGO_LIN', 'LINHA', 'MAX_ALIMEN', 'ALIMENTADO', 'LONG',
       'LAT', 'Q__z', 'geometry'],
      dtype='object')

In [85]:
gdf_ramal.head()

,CODIGO,COMPRIMENT,COMPRIME00,FASES,TIPO_INSTA,RAMAL_SERV,PROPRIETAR,MAX_CIRCUI,MAX_MODELO,NOME,...,A5_NEUTRO,A6_NEUTRO,CODIGO_LIN,LINHA,MAX_ALIMEN,ALIMENTADO,LONG,LAT,Q__z,geometry
0,134701821.0,12.8186,12.8186,ABCN,Aéreo,Sim,Light,139791206,105925093,ZNA800946,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.474014,-22.757567,NaN,"LINESTRING (-43.47401 -22.75757, -43.47397 -22..."
1,134701828.0,13.0483,13.0483,ABCN,Aéreo,Sim,Light,27041984,105925093,ZNA853915,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.466650,-22.750844,NaN,"LINESTRING (-43.46665 -22.75084, -43.46656 -22..."
2,134701835.0,12.0416,13.6142,ABCN,Aéreo,Sim,Light,27042119,105925094,ZNA853228,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.464115,-22.750439,NaN,"LINESTRING (-43.46411 -22.75044, -43.46411 -22..."
3,134701842.0,8.1876,8.1876,ABCN,Aéreo,Sim,Light,27060045,105925094,ZNA14011,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.463247,-22.750807,NaN,"LINESTRING (-43.46325 -22.75081, -43.46331 -22..."
4,134701849.0,10.0333,10.0333,CN,Aéreo,Sim,Light,133462054,105925094,ZNA801319,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.472297,-22.747533,NaN,"LINESTRING (-43.47230 -22.74753, -43.47223 -22..."


In [86]:
gdf_ramal['LINHA'].value_counts()

LINHA
LDA TAMOIO IV    7431
LDA VIENA        6329
LDA VIRGILIO     6198
LDA BERLIM       6049
LDA HAVANA       6014
                 ... 
LDA RIO 2           3
LDS 4191            2
LDA GIBRAN          1
LSA RODUTRA         1
LDA OURO            1
Name: count, Length: 288, dtype: int64

In [88]:
gdf_ramal['CODIGO'] = gdf_ramal['CODIGO'].astype(int)
gdf_ramal['COMPRIMENT'] = gdf_ramal['COMPRIMENT'].astype(float)

In [90]:
gdf_ramal.head()

,CODIGO,COMPRIMENT,COMPRIME00,FASES,TIPO_INSTA,RAMAL_SERV,PROPRIETAR,MAX_CIRCUI,MAX_MODELO,NOME,...,A5_NEUTRO,A6_NEUTRO,CODIGO_LIN,LINHA,MAX_ALIMEN,ALIMENTADO,LONG,LAT,Q__z,geometry
0,134701821,12.8186,12.8186,ABCN,Aéreo,Sim,Light,139791206,105925093,ZNA800946,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.474014,-22.757567,NaN,"LINESTRING (-43.47401 -22.75757, -43.47397 -22..."
1,134701828,13.0483,13.0483,ABCN,Aéreo,Sim,Light,27041984,105925093,ZNA853915,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.466650,-22.750844,NaN,"LINESTRING (-43.46665 -22.75084, -43.46656 -22..."
2,134701835,12.0416,13.6142,ABCN,Aéreo,Sim,Light,27042119,105925094,ZNA853228,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.464115,-22.750439,NaN,"LINESTRING (-43.46411 -22.75044, -43.46411 -22..."
3,134701842,8.1876,8.1876,ABCN,Aéreo,Sim,Light,27060045,105925094,ZNA14011,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.463247,-22.750807,NaN,"LINESTRING (-43.46325 -22.75081, -43.46331 -22..."
4,134701849,10.0333,10.0333,CN,Aéreo,Sim,Light,133462054,105925094,ZNA801319,...,00,00,55513509.0,LDA THOMAZ,55513488,LDA THOMAZ,-43.472297,-22.747533,NaN,"LINESTRING (-43.47230 -22.74753, -43.47223 -22..."


In [91]:
# Agrupar por 'Categoria' e calcular a soma e a contagem
BI_ramal= gdf_ramal.groupby('LINHA').agg({'COMPRIMENT': ['count', 'sum']}).reset_index()

# Renomear as colunas
BI_ramal.columns = ['LINHA', 'Ramal_servico_qtd', 'Ramal_servico_km']

BI_ramal

,LINHA,Ramal_servico_qtd,Ramal_servico_km
0,LDA ABAETE,1226,19584.6663
1,LDA ABATOS,208,2853.9133
2,LDA ABELARDO,48,571.1763
3,LDA ACHE,641,8214.0757
4,LDA ADUTORA,806,10922.8200
...,...,...,...
283,LRA R CARLEME,178,2522.1462
284,LRA R PAÇUCAR,97,1233.5661
285,LRA R POLIXENA,32,463.8406
286,LRA R SEVERIANO,5,63.9105


# Poste

In [111]:
df_poste = pd.read_excel(path_ext + extracao + "/POSTES_LINHA_unificado.xlsx", sheet_name= "Planilha1" )
df_poste.columns = df_poste.iloc[1]

# Descartar as primeiras duas linhas
df_poste = df_poste.drop([0, 1
                          ])

# Resetar o índice
df_poste = df_poste.reset_index(drop=True)

df_poste.columns = ['ALIMENTADOR', 'Poste_qtd']


In [112]:
df_escopo

,ITEM,REGIONAL,ALIMENTADOR,SUBESTAÇÃO,POSTE,TRAFO,RELIGAD.,SECCION.,KM_MT,KM_BT,KM_RAMAIS,QTDE RAMAIS,CLIENTES BT
0,273,OESTE,LDS 61297,SETD GARDENIA,12,4,0,0,3.264561,0,0.062183,14,NaN
1,340,OESTE,LDA BANDER,SETD CURICICA,12,0,0,0,2.694859,0,0,0,NaN
2,341,OESTE,LDS 61299,SETD GARDENIA,12,0,0,0,4.924825,0,0,0,NaN
3,339,OESTE,LDA NOBET,SETD JABOATAO,13,0,0,0,2.008086,0,0,0,NaN
4,275,OESTE,LDS 61139,SETD GARDENIA,16,4,0,0,3.172677,0.022557,0.077588,11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,119,BAIXADA,LDA TABOLEIRO,SESD TUIUTI,1492,138,4,5,54.869917,43.22154,24.541077,1614,NaN
335,284,BAIXADA,LDA HORTENCIA/CENTAUREA,SETD COMENDADOR SOARES,1517,76,3,2,0,0,0,0,NaN
336,96,OESTE,LDA VILAMIL,SETD PADRE MIGUEL,1553,135,5,5,32.474668,34.030487,26.744972,1986,NaN
337,2,CENTRO SUL,LDA RIOMORTO,SETD RECREIO,1645,152,4,10,31.790495,56.096736,67.980405,6316,NaN


In [115]:
df_poste_escopo = df_poste.merge(df_escopo[['ALIMENTADOR']], how = 'outer', on = "ALIMENTADOR", indicator = True)   

In [123]:
df_poste_escopo[df_poste_escopo['_merge'] == 'right_only']

,ALIMENTADOR,Poste_qtd,_merge
1827,LDS 61297,NaN,right_only
1828,LDS 61299,NaN,right_only
1829,LDS 61139,NaN,right_only
1830,LDS 30534,NaN,right_only
1831,LDS 61265,NaN,right_only
1832,LDS 60819,NaN,right_only
1833,LDS 61906,NaN,right_only
1834,LDS 61805,NaN,right_only
1835,LDS 61704,NaN,right_only
1836,LDS 61016,NaN,right_only


In [118]:
df_poste_escopo[df_poste_escopo['_merge'] == 'right_only'].to_csv(path_resultados + "Postes do escopo nao encontrados.csv", sep = ';')

In [121]:
df_poste_escopo_both = df_poste_escopo[df_poste_escopo['_merge'] == 'both']

In [127]:
BI_Poste = df_poste_escopo_both[['ALIMENTADOR', 'Poste_qtd']]
BI_Poste.columns = ['LINHA', 'Poste_qtd']

In [128]:
BI_Poste

,LINHA,Poste_qtd
0,LDA ABAETE,292
4,LDA ABATOS,261
6,LDA ABELARDO,131
16,LDA ADUTORA,1008
25,LDA AGUIA,665
...,...,...
1793,LSA RODUTRA,666
1795,LSA SABIA,228
1799,LSA SAMPAIO,97
1805,LSA SHALON,544


# Compilar BI

In [159]:
BI_Comp = BI_trecho.merge(BI_trafo, how = 'left').merge(BI_Poste, how = 'left').merge(BI_ramal, how = 'left').fillna(0)

In [160]:
BI_Comp

,LINHA,REDE_MT_KM,qtd_trafo,Poste_qtd,Ramal_servico_qtd,Ramal_servico_km
0,LDA ABAETE,4801.0447,29.0,292,1226.0,19584.6663
1,LDA ABATOS,17277.1451,50.0,261,208.0,2853.9133
2,LDA ABELARDO,4193.9757,23.0,131,48.0,571.1763
3,LDA ACHE,5965.3535,45.0,0,641.0,8214.0757
4,LDA ADUTORA,59927.2544,114.0,1008,806.0,10922.8200
...,...,...,...,...,...,...
334,LSA RODUTRA,34739.5379,0.0,666,1.0,5.0057
335,LSA SABIA,9113.7148,0.0,228,0.0,0.0000
336,LSA SAMPAIO,2833.5922,0.0,97,0.0,0.0000
337,LSA SHALON,22343.2989,0.0,544,0.0,0.0000


In [161]:
BI_Comp = BI_Comp[['LINHA','Poste_qtd', 'qtd_trafo','Ramal_servico_qtd','Ramal_servico_km', 'REDE_MT_KM' ]]

In [163]:
BI_Comp.round(0).to_csv(path_resultados + extracao + '_Ativos_BI_gerencial.csv', sep = ';')

## Check info escopo

In [141]:
check_shp_escopo = df_escopo[['ALIMENTADOR', 'POSTE', 'TRAFO', 'KM_MT', 'KM_RAMAIS','QTDE RAMAIS']].merge(BI_Comp, how = 'left', right_on = 'LINHA', left_on = 'ALIMENTADOR').fillna(0)

In [144]:
check_shp_escopo

,ALIMENTADOR,POSTE,TRAFO,KM_MT,KM_RAMAIS,QTDE RAMAIS,LINHA,REDE_MT_KM,qtd_trafo,Poste_qtd,Ramal_servico_qtd,Ramal_servico_km
0,LDS 61297,12,4,3.264561,0.062183,14,0,0.0000,0.0,0.0,0.0,0.0000
1,LDA BANDER,12,0,2.694859,0.000000,0,LDA BANDER,2724.5015,0.0,81.0,0.0,0.0000
2,LDS 61299,12,0,4.924825,0.000000,0,0,0.0000,0.0,0.0,0.0,0.0000
3,LDA NOBET,13,0,2.008086,0.000000,0,LDA NOBET,1414.7733,0.0,50.0,0.0,0.0000
4,LDS 61139,16,4,3.172677,0.077588,11,0,0.0000,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
334,LDA TABOLEIRO,1492,138,54.869917,24.541077,1614,LDA TABOLEIRO,55241.0909,134.0,1530.0,1592.0,55617.8024
335,LDA HORTENCIA/CENTAUREA,1517,76,0.000000,0.000000,0,0,0.0000,0.0,0.0,0.0,0.0000
336,LDA VILAMIL,1553,135,32.474668,26.744972,1986,LDA VILAMIL,31464.1699,86.0,1642.0,1964.0,28572.6953
337,LDA RIOMORTO,1645,152,31.790495,67.980405,6316,LDA RIOMORTO,23900.6292,107.0,1262.0,4063.0,64266.4546


In [146]:
check_shp_escopo['DIF_Poste'] = check_shp_escopo['POSTE'] - check_shp_escopo['Poste_qtd']
check_shp_escopo['DIF_TRAFO'] = check_shp_escopo['TRAFO'] - check_shp_escopo['qtd_trafo']
check_shp_escopo['DIF_KM_REDE'] = check_shp_escopo['KM_MT'] - check_shp_escopo['REDE_MT_KM']
check_shp_escopo['DIF_KM_RAMAL'] = check_shp_escopo['KM_RAMAIS'] - check_shp_escopo['Ramal_servico_km']
check_shp_escopo['DIF_QTD_RAMAL'] = check_shp_escopo['QTDE RAMAIS'] - check_shp_escopo['Ramal_servico_qtd']

In [151]:
check_shp_escopo[['ALIMENTADOR','LINHA','DIF_Poste','DIF_TRAFO','DIF_KM_REDE','DIF_KM_RAMAL', 'DIF_QTD_RAMAL']]

,ALIMENTADOR,LINHA,DIF_Poste,DIF_TRAFO,DIF_KM_REDE,DIF_KM_RAMAL,DIF_QTD_RAMAL
0,LDS 61297,0,12.0,4.0,3.264561,0.062183,14.0
1,LDA BANDER,LDA BANDER,-69.0,0.0,-2721.806641,0.000000,0.0
2,LDS 61299,0,12.0,0.0,4.924825,0.000000,0.0
3,LDA NOBET,LDA NOBET,-37.0,0.0,-1412.765214,0.000000,0.0
4,LDS 61139,0,16.0,4.0,3.172677,0.077588,11.0
...,...,...,...,...,...,...,...
334,LDA TABOLEIRO,LDA TABOLEIRO,-38.0,4.0,-55186.220984,-55593.261323,22.0
335,LDA HORTENCIA/CENTAUREA,0,1517.0,76.0,0.000000,0.000000,0.0
336,LDA VILAMIL,LDA VILAMIL,-89.0,49.0,-31431.695232,-28545.950327,22.0
337,LDA RIOMORTO,LDA RIOMORTO,383.0,45.0,-23868.838705,-64198.474195,2253.0


In [ ]:
gdf_TRAFO = gpd.read_file(path_ext +extracao+ "/TRANSFORMADOR.shp")
gdf_TRAFO['geometry'] = gdf_TRAFO['geometry'].to_crs(4326)

In [ ]:
gdf_TRAFO = gdf_TRAFO.set_geometry('geometry')

In [ ]:
gdf_TRAFO.columns

Index(['CODIGO', 'MATRICULA', 'CAPAC_CHAV', 'TIPO_INSTA', 'EXCLUSIVO_',
       'FASES', 'TIPO_TRANS', 'TIPO_LIGAC', 'TIPO_ATERR', 'POSTO',
       'TIPO_MONTA', 'CAPAC_ELO', 'AUTOPROTEG', 'BLINDADO', 'MAX_LINHA',
       'MAX_EQTRAN', 'LOTE', 'MATRICULA_', 'LP', 'CLASSE', 'FABRICANTE',
       'DATA_FABR', 'SERIE', 'QTDE_FASES', 'POTENCIA_N', 'TENSAO_PRI',
       'TENSAO_SEC', 'MEIO_ISOLA', 'MAT_ENROLA', 'MAX_TRANSF', 'CODIGO_LIN',
       'LINHA', 'MAX_ALIMEN', 'NOME', 'ALIMENTADO', 'LONG', 'LAT', 'Q__z',
       'geometry'],
      dtype='object')

In [ ]:
gdf_TRAFO.head()

,CODIGO,MATRICULA,CAPAC_CHAV,TIPO_INSTA,EXCLUSIVO_,FASES,TIPO_TRANS,TIPO_LIGAC,TIPO_ATERR,POSTO,...,MAX_TRANSF,CODIGO_LIN,LINHA,MAX_ALIMEN,NOME,ALIMENTADO,LONG,LAT,Q__z,geometry
0,748478984.0,ZNA437651,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,748478984,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.699292,-22.926614,NaN,POINT (-43.69929 -22.92661)
1,19746023.0,ZNA434125,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19746023,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.691641,-22.922198,NaN,POINT (-43.69164 -22.92220)
2,19719449.0,ZNA44040,100A,Aéreo,Não,ABC,Banco trifásico paralelo,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19719449,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.701437,-22.925699,NaN,POINT (-43.70144 -22.92570)
3,19719449.0,ZNA44040,100A,Aéreo,Não,ABC,Banco trifásico paralelo,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19719449,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.701437,-22.925699,NaN,POINT (-43.70144 -22.92570)
4,620617.0,ZNA28024,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,620617,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372470,-22.886976,NaN,POINT (-43.37247 -22.88698)


In [ ]:
gdf_TRAFO['LINHA'].value_counts()

LINHA
LDA JEQUITIBA      175
LDA SANTAREM       171
LDA BELAVI         167
LDA GADOBAN        147
LDA RODU           147
                  ... 
LDA RIO 2            3
LDA RESERVA-ZIN      2
LDA OURO             2
LDA SALVALENDE       1
LDA GIBRAN           1
Name: count, Length: 287, dtype: int64

In [ ]:
gdf_TRAFO['CODIGO'] = gdf_trecho_mt['CODIGO'].astype(int)
#gdf_trecho_mt['COMPRIMENT'] = gdf_trecho_mt['COMPRIMENT'].astype(float)

In [ ]:
gdf_TRAFO.head()

,CODIGO,MATRICULA,CAPAC_CHAV,TIPO_INSTA,EXCLUSIVO_,FASES,TIPO_TRANS,TIPO_LIGAC,TIPO_ATERR,POSTO,...,MAX_TRANSF,CODIGO_LIN,LINHA,MAX_ALIMEN,NOME,ALIMENTADO,LONG,LAT,Q__z,geometry
0,617676,ZNA437651,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,748478984,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.699292,-22.926614,NaN,POINT (-43.69929 -22.92661)
1,617700,ZNA434125,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19746023,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.691641,-22.922198,NaN,POINT (-43.69164 -22.92220)
2,653264,ZNA44040,100A,Aéreo,Não,ABC,Banco trifásico paralelo,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19719449,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.701437,-22.925699,NaN,POINT (-43.70144 -22.92570)
3,653360,ZNA44040,100A,Aéreo,Não,ABC,Banco trifásico paralelo,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,19719449,18796580.0,LDA MARICA,18772547,LDA MARICA,LDA MARICA,-43.701437,-22.925699,NaN,POINT (-43.70144 -22.92570)
4,653390,ZNA28024,100A,Aéreo,Não,ABC,Transformador de distribuição,Delta - Estrela,Diretamente aterrado,Zona de Distribuição Aérea,...,620617,615100.0,LDA MAFRA,613489,LDA MAFRA,LDA MAFRA,-43.372470,-22.886976,NaN,POINT (-43.37247 -22.88698)


In [ ]:
# Agrupar por 'Categoria' e calcular a soma e a contagem
BI_trafo= gdf_TRAFO.groupby('LINHA').agg({'CODIGO': ['count']}).reset_index()

# Renomear as colunas
BI_trafo.columns = ['LINHA', 'qtd_trafo']

BI_trafo

,LINHA,qtd_trafo
0,LDA ABAETE,29
1,LDA ABATOS,50
2,LDA ABELARDO,23
3,LDA ACHE,45
4,LDA ADUTORA,114
...,...,...
282,LRA PEIXOTO,8
283,LRA R CARLEME,23
284,LRA R PAÇUCAR,7
285,LRA R POLIXENA,7


# leitura ar

In [15]:
pasta_AR='D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/8. Área de risco/0.4 - Poligonais áreas de risco/'
file_suffixes = ['ACAC.json', 'ASRO.geojson', 'COMUNIDADES_DOMINADAS_PELA_MILICIA.geojson', 'COMUNIDADES_DOMINADAS_PELO_AMIGOS_DOS_AMIGOS.geojson', 'COMUNIDADES_DOMINADAS_PELO_COMANDO_VEMELHO.geojson', 'COMUNIDADES_DOMINADAS_PELO_TERCEIRO_COMANDO_PURO.geojson']

In [16]:
shp_AR0 =gpd.read_file(pasta_AR +'ACAC.json')
shp_AR1 =gpd.read_file(pasta_AR +'ASRO.geojson')
shp_AR2 =gpd.read_file(pasta_AR +'COMUNIDADES_DOMINADAS_PELA_MILICIA.geojson')
shp_AR3 =gpd.read_file(pasta_AR +'COMUNIDADES_DOMINADAS_PELO_AMIGOS_DOS_AMIGOS.geojson')
shp_AR4 =gpd.read_file(pasta_AR +'COMUNIDADES_DOMINADAS_PELO_COMANDO_VEMELHO.geojson')
shp_AR5 =gpd.read_file(pasta_AR +'COMUNIDADES_DOMINADAS_PELO_TERCEIRO_COMANDO_PURO.geojson')

#### Library

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
from os import listdir
import zipfile
import time
from datetime import datetime
from shapely.ops import unary_union
# import openpyxl
import glob
import pickle
# import pygeos
from shapely.geometry import Point, LineString
import folium
import pyproj
# import cv2
import matplotlib.pyplot as plt
from folium import plugins
import xlsxwriter



#import folium
#from folium import plugins
#from folium import Map, FeatureGroup, Marker, LayerControl, Popup
#from folium.features import DivIcon
import branca.colormap as cm

pd.options.display.max_rows = 10

#import atrb_mcpse

#### function

In [2]:
# Mapas
# Add custom base maps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps'#,
        #overlay = True,
        #control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite'#,
        #overlay = True,
        #control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain'#,
        #overlay = True,
        #control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite'#,
        #overlay = True,
        #control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite'#,
        #overlay = True,
        #control = True
    ),
    'CartoDB positron': folium.TileLayer(
        tiles = 'CartoDB positron',
        attr = 'CartoDB',
        name = 'Light Map'#,
        #overlay = True,
        #control = True
    )  
 }

def mapa(df, indicadores, df2): 
    
    df_colormap = cm.LinearColormap(
    ['blue', 'red'],
     vmin= 0, vmax=1,    
    caption= indicadores + ' Color Scale' #Caption for Color scale or Legend
    )
   ### Centroides Localidades 
    sim_geo = gpd.GeoSeries(df['geometry'])
    geo_json_data = df.to_json()
    #centroides = df.iloc[[0]].to_crs(22182).centroid.to_crs(4326)
    #centroide=df.iloc[[0]].to_crs(22182).centroid.to_crs(4326)
    centroides = df.iloc[[0]].centroid
    centroide=df.iloc[[0]].centroid

    mapa = folium.Map([centroide.y,centroide.x ],
                      zoom_start=8.3,
                      width=800, height=600,
                      tiles=None        )
    
    
    
        # Add custom basemaps
    basemaps['Google Maps'].add_to(mapa)
    basemaps['Google Satellite Hybrid'].add_to(mapa)
    basemaps['Google Satellite'].add_to(mapa)
    basemaps['Google Terrain'].add_to(mapa)
    basemaps['Esri Satellite'].add_to(mapa)
    basemaps['CartoDB positron'].add_to(mapa)
    folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mapa)
    
     


    style_function = lambda feature: {
           'fillColor': df_colormap(feature['properties'][indicadores]),
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.3
                }

    
    folium.GeoJson(
        geo_json_data,
        style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
                fields = [indicadores],
                localize=False)

        ).add_to(mapa)
    
    
    if len(df2) >0:
        style_function = lambda feature: {
            'fillColor': 'red',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.3
                }
        folium.GeoJson(df2, style_function=style_function).add_to(mapa)
    
    folium.map.CustomPane('labels').add_to(mapa)
    folium.TileLayer('CartoDBPositronOnlyLabels',
                     pane='labels').add_to(mapa)

    #fullscreen
    plugins.Fullscreen().add_to(mapa)

    df_colormap.add_to(mapa) 
    # Add a layer control panel to the map.
    #mapa.add_child(folium.LayerControl())
    folium.LayerControl().add_to(mapa)   
    
    return mapa

def mapa_atrib(df, label,df2): 

    ### Centroides Localidades 
    sim_geo = gpd.GeoSeries(df['geometry'])
    geo_json_data = df.to_json()
    df['x']=df.centroid.x
    df['y']=df.centroid.y
    centroides = df.iloc[[0]].centroid
    centroide=df.iloc[[0]].centroid
    
    mapa = folium.Map([centroide.y,centroide.x ],
                      zoom_start=14,
                      width=800, height=600,
                      tiles=None        )
    minimap = plugins.MiniMap(toggle_display=True)
    mapa.add_child(minimap)
    #plugins.ScrollZoomToggler().add_to(mapa)
    plugins.Fullscreen(position="topright").add_to(mapa)
    
        # Add custom basemaps
    basemaps['Google Maps'].add_to(mapa)
    basemaps['Google Satellite Hybrid'].add_to(mapa)
    basemaps['Google Satellite'].add_to(mapa)
    basemaps['Google Terrain'].add_to(mapa)
    basemaps['Esri Satellite'].add_to(mapa)
    basemaps['CartoDB positron'].add_to(mapa)
    folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mapa)
    
    
    style_function = lambda feature: {
        'fillColor': 'red',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.3
            }
    folium.GeoJson(df, style_function=style_function).add_to(mapa)
    folium.GeoJson(df2).add_to(mapa)
    # add marker one by one on the map
    for i in range(0,len(df)):
       folium.Marker(
          location=[df.iloc[i]['y'],df.iloc[i]['x']],
          popup=df.iloc[i][label],
          icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: blue">{df.iloc[i][label]}</div>""")
       ).add_to(mapa)

    return mapa


### Path and read

In [18]:
path_fme = "D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME"
# path_res = "D:/2. Light_2023/Inventário/4. Resultados/2. Calculadora/"
# path_excel = "D:/2. Light_2023/Inventário/4. Resultados/2. Calculadora/Circuitos/"

In [19]:
path_fme

'D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME'

In [20]:
import os
import glob
import fiona

def listar_arquivos_shp(path_fme):
    # Obtém todos os arquivos .shp na pasta fornecida
    padrao = os.path.join(path_fme, '*.shp')
    arquivos_shp = glob.glob(padrao)

    # Imprime o caminho de cada arquivo .shp
    for arquivo_shp in arquivos_shp:
        print(arquivo_shp)

    # Informa se não houver arquivos .shp na pasta
    if not arquivos_shp:
        print("Nenhum arquivo .shp encontrado na pasta.")

if __name__ == "__main__":
    # Substitua 'caminho/para/sua/pasta' pelo caminho da pasta desejada
    caminho_da_pasta = path_fme
    listar_arquivos_shp(caminho_da_pasta)


D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME\Banco de Capacitores.shp
D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME\Fly Tap BT.shp
D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME\Fly Tap.shp
D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME\Transformador.shp
D:/GIT/2022_23_LIGHT_Sust_Concessao/4. Execução/4. Inventário de campo/2. Informações/5. Amostragem/1. FME/Extrações no FME\Trecho MT.shp


In [21]:
Trecho_MT = gpd.read_file(path_fme + "/Trecho MT.shp")

# Banco de Capacitores

In [8]:
BC = gpd.read_file(path_fme + "/Banco de Capacitores.shp")

In [10]:
BC.columns

Index(['CODIGO', 'MATRICULA', 'NUM_UNIDAD', 'NUM_UNID00', 'CLASSE_TEN',
       'POTENCIA', 'TIPO_CONTR', 'SUB_TIPO_C', 'TIPO_LIGAC', 'CORRENTE_N',
       'CAP_NOMINA', 'ESTADO', 'FASES', 'MAX_LINHA', 'LOTE', 'NOME_LINHA',
       'LONG', 'LAT', 'Q__z', 'geometry'],
      dtype='object')

In [15]:
BC

,CODIGO,MATRICULA,NUM_UNIDAD,NUM_UNID00,CLASSE_TEN,POTENCIA,TIPO_CONTR,SUB_TIPO_C,TIPO_LIGAC,CORRENTE_N,CAP_NOMINA,ESTADO,FASES,MAX_LINHA,LOTE,NOME_LINHA,LONG,LAT,Q__z,geometry
0,1646082.0,BC81417,3.0,3.0,"15,0 kV",300 KVAR,Direto,FIXO,Estrela,NaN,50.0,Ligado,ABC,615104,NaN,LDA MALOCA,-43.264677,-22.869311,NaN,POINT (-43.26468 -22.86931)
1,1646087.0,BC95011,NaN,NaN,"15,0 kV",300 KVAR,Direto,FIXO,Delta,NaN,50.0,Ligado,ABC,615343,NaN,LDA DONABEL,-43.253919,-22.870021,NaN,POINT (-43.25392 -22.87002)
2,1646092.0,BC81466,NaN,NaN,"15,0 kV",300 KVAR,Direto,FIXO,Delta,NaN,50.0,Ligado,ABC,615343,NaN,LDA DONABEL,-43.249259,-22.869220,NaN,POINT (-43.24926 -22.86922)
3,1646097.0,BC81423,NaN,NaN,"15,0 kV",300 KVAR,Direto,FIXO,Estrela,NaN,50.0,Ligado,ABC,615174,NaN,LDA PROCLAMACAO,-43.255087,-22.869407,NaN,POINT (-43.25509 -22.86941)
4,1646102.0,BC95010,NaN,NaN,"15,0 kV",300 KVAR,Direto,FIXO,Estrela,NaN,100.0,Ligado,ABC,615286,NaN,LDA BERGAMINI,-43.296459,-22.904189,NaN,POINT (-43.29646 -22.90419)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,10935793.0,BC85773,1.0,3.0,"15,0 kV",300 KVAR,Direto,MANUAL,Estrela,NaN,50.0,Ligado,ABC,496893099,NaN,LDA BUENO,-43.351127,-22.961184,NaN,POINT (-43.35113 -22.96118)
285,18945350.0,BC534278,3.0,3.0,"15,0 kV",300 KVAR,Direto,MANUAL,Estrela,NaN,50.0,Ligado,ABC,18796561,NaN,LDA MACONARIA,-43.204100,-22.111362,NaN,POINT (-43.20410 -22.11136)
286,32389758.0,BC4397,NaN,NaN,"15,0 kV",300 KVAR,Direto,MANUAL,Estrela,NaN,300.0,Ligado,ABC,18796622,NaN,LDA MORADA,-44.114825,-22.540969,NaN,POINT (-44.11482 -22.54097)
287,1646752.0,BC95008,3.0,3.0,"15,0 kV",300 KVAR,Direto,FIXO,Estrela,NaN,50.0,Ligado,ABC,615125,NaN,LDA MELGACO,-43.295522,-22.816600,NaN,POINT (-43.29552 -22.81660)


# Fly Tap BT

In [12]:
FT_BT = gpd.read_file(path_fme + "/Fly Tap BT.shp")

In [14]:
FT_BT.columns

Index(['CODIGO', 'MATRICULA', 'MAX_CIRCUI', 'NOME', 'MAX_TRANSF', 'MAX_LINHA',
       'NOME_LINHA', 'LONG', 'LAT', 'Q__z', 'geometry'],
      dtype='object')

In [16]:
FT_BT['NOME_LINHA'].value_counts()

NOME_LINHA
LDA CURIE          20
LDA ALMIRANTE      18
LDA AMAZONAS       13
LDA SIDEROPOLIS    12
LDA CICUTA         12
                   ..
LDA ZOOTECNICO      1
LDA CERAMICA        1
LDA RICA            1
LDA ITAJAI          1
LDA GURUPI          1
Name: count, Length: 78, dtype: int64

# Fly Tap

In [ ]:
FT = gpd.read_file(path_fme + "/Fly Tap.shp")

In [ ]:
FT.columns

Index(['CODIGO', 'MATRICULA', 'MAX_CIRCUI', 'NOME', 'MAX_TRANSF', 'MAX_LINHA',
       'NOME_LINHA', 'LONG', 'LAT', 'Q__z', 'geometry'],
      dtype='object')

In [ ]:
FT['NOME_LINHA'].value_counts()

NOME_LINHA
LDA CURIE          20
LDA ALMIRANTE      18
LDA AMAZONAS       13
LDA SIDEROPOLIS    12
LDA CICUTA         12
                   ..
LDA ZOOTECNICO      1
LDA CERAMICA        1
LDA RICA            1
LDA ITAJAI          1
LDA GURUPI          1
Name: count, Length: 78, dtype: int64

# Trecho MT

In [17]:
Trecho_MT = gpd.read_file(path_fme + "/Trecho MT.shp")

NameError: name 'path_fme' is not defined

In [3]:
Trecho_MT['NOME_LINHA'].value_counts().describe()

NameError: name 'Trecho_MT' is not defined

# Transformador

In [4]:
TRAFO = gpd.read_file(path_fme + "/Transformador.shp")

NameError: name 'path_fme' is not defined

In [21]:
TRAFO['NOME_LINHA'].value_counts()

NOME_LINHA
LDA SANTAREM / SERV.    270
LDA JEQUITIBA           175
LDA BELAVI              168
LDA GADOBAN             153
LDA RODU                147
                       ... 
LDA OURO                  2
LDS 61704                 1
LDA DAGRACA               1
LDA ILHAPURA              1
LDA SALVALENDE            1
Name: count, Length: 295, dtype: int64

In [9]:
alimentador = CTMT[CTMT['COD_ID'].isin(NOME_CT)]
alimentador = alimentador[['COD_ID', 'BARR', 'SUB', 'PAC_INI', 'TEN_NOM','NOME']]
alimentador

,COD_ID,BARR,SUB,PAC_INI,TEN_NOM,NOME


In [10]:
CTMT.to_csv('CTMT.csv', sep = ';')

# SEGMENTOS BT

In [11]:
SSDBT =   pd.read_pickle(path_info + 'SSDBT.pkl')
SSDBT.columns

Index(['COD_ID', 'PN_CON_1', 'PN_CON_2', 'UNI_TR_MT', 'CTMT', 'CT_COD_OP',
       'UNI_TR_AT', 'SUB', 'CONJ', 'ARE_LOC', 'FAS_CON', 'DIST', 'PAC_1',
       'PAC_2', 'TIP_INST', 'TIP_CND', 'POS', 'ODI', 'TI', 'CM', 'SITCONT',
       'COMP', 'DESCR', 'geometry', 'file', 'TIP_INST_dic', 'len'],
      dtype='object')

In [12]:
df_SSDBT = SSDBT[SSDBT['CTMT'].isin(alimentador['COD_ID'])]


In [13]:
df_SSDBT['CTMT'].value_counts()

Series([], Name: count, dtype: int64)

### Distâncias (m) e Quantidades

In [14]:
# colunas_agrupadas = ['DIST','CTMT']
# colunas_agrupadas = ['DIST','CTMT', 'FAS_CON']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_INST_dic' ]
# colunas_agrupadas = ['DIST','CTMT', 'TIP_CND']
colunas_agrupadas = ['DIST','CTMT','TIP_INST_dic', 'FAS_CON']



In [15]:
colunas_agrupadas[1:]

['CTMT', 'TIP_INST_dic', 'FAS_CON']

In [16]:
check_BT = df_SSDBT.groupby(colunas_agrupadas[1]).agg({'DIST': 'count', 'COMP': 'sum' ,'len' : 'sum' }).reset_index()

In [17]:
df_SSDBT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count', 'COMP': 'sum' ,'len' : 'sum' }).reset_index()

,CTMT,TIP_INST_dic,FAS_CON,DIST,COMP,len


### SEGCON

In [18]:
SEGCON = pd.read_pickle(path_info + "SEGCON.pkl")
SEGCON.columns


Index(['COD_ID', 'DIST', 'GEOM_CAB', 'FORM_CAB', 'BIT_FAS_1', 'BIT_FAS_2',
       'BIT_FAS_3', 'BIT_NEU', 'MAT_FAS_1', 'MAT_FAS_2', 'MAT_FAS_3',
       'MAT_NEU', 'ISO_FAS_1', 'ISO_FAS_2', 'ISO_FAS_3', 'ISO_NEU', 'CND_FAS',
       'R1', 'X1', 'R_REGUL', 'FTRCNV', 'CNOM', 'CMAX', 'TUC_FAS', 'A1_FAS',
       'A2_FAS', 'A3_FAS', 'A4_FAS', 'A5_FAS', 'A6_FAS', 'TUC_NEU', 'A1_NEU',
       'A2_NEU', 'A3_NEU', 'A4_NEU', 'A5_NEU', 'A6_NEU', 'UAR', 'DESCR',
       'geometry', 'file', 'GEOM_CAB_dic', 'FORM_CAB_dic', 'BIT_FAS_1_dic',
       'BIT_FAS_2_dic', 'BIT_FAS_3_dic', 'BIT_NEU_dic', 'MAT_FAS_1_dic',
       'MAT_FAS_2_dic', 'MAT_FAS_3_dic', 'MAT_NEU_dic', 'ISO_FAS_1_dic',
       'ISO_FAS_2_dic', 'ISO_FAS_3_dic', 'ISO_NEU_dic', 'TUC_FAS_dic',
       'A1_FAS_dic', 'A2_FAS_dic', 'A3_FAS_dic', 'A4_FAS_dic', 'A5_FAS_dic',
       'A6_FAS_dic', 'TUC_NEU_dic', 'A1_NEU_dic', 'A2_NEU_dic', 'A3_NEU_dic',
       'A4_NEU_dic', 'A5_NEU_dic', 'A6_NEU_dic'],
      dtype='object')

In [19]:
#trocar
## DIST - quantidade
## len - Geo_comp

## Tucagem
# colunas_agrupadas_segcon = ['COD_ID','TUC_FAS', 'A1_FAS','A2_FAS','A3_FAS', 'A4_FAS', 'A5_FAS', 'A6_FAS']

## Personalizado
# colunas_agrupadas_segcon = ['COD_ID','GEOM_CAB_dic','FORM_CAB','FORM_CAB_dic', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']
colunas_agrupadas_segcon = ['COD_ID','GEOM_CAB_dic', 'FORM_CAB_dic',                                # geometria e formação    
                            'BIT_FAS_1_dic','BIT_FAS_2_dic','BIT_FAS_3_dic','BIT_NEU_dic',         # bitola
                            'MAT_FAS_1_dic','MAT_FAS_2_dic', 'MAT_FAS_3_dic', 'MAT_NEU_dic',    # material
                            'ISO_FAS_1_dic', 'ISO_FAS_2_dic', 'ISO_FAS_3_dic', 'ISO_NEU_dic',                    # isolador 
                            'TUC_FAS_dic', 'A1_FAS_dic', 'A2_FAS_dic', 'A3_FAS_dic', 'A4_FAS_dic', 'A5_FAS_dic', 'A6_FAS_dic',  # Tuc Fase
                            'TUC_NEU_dic', 'A1_NEU_dic','A2_NEU_dic', 'A3_NEU_dic', 'A4_NEU_dic', 'A5_NEU_dic', 'A6_NEU_dic',   # Tuc Neutro                         
                            'CND_FAS']                                                              # Condutor por fase    

In [20]:
sum_SEG_BT = df_SSDBT.merge(SEGCON[colunas_agrupadas_segcon], left_on = 'TIP_CND', right_on = 'COD_ID', how ='left').groupby(colunas_agrupadas[1:] +colunas_agrupadas_segcon[1:] ).agg({'DIST': 'count', 'COMP': 'sum' ,'len' : 'sum' }).reset_index()
sum_SEG_BT.rename(columns={'DIST': 'QUANTIDADE', 'len': 'GEO_COMP'}, inplace=True)
sum_SEG_BT['NIVEL_TBL'] = 'BT'
sum_SEG_BT

,CTMT,TIP_INST_dic,FAS_CON,GEOM_CAB_dic,FORM_CAB_dic,BIT_FAS_1_dic,BIT_FAS_2_dic,BIT_FAS_3_dic,BIT_NEU_dic,MAT_FAS_1_dic,...,A2_NEU_dic,A3_NEU_dic,A4_NEU_dic,A5_NEU_dic,A6_NEU_dic,CND_FAS,QUANTIDADE,COMP,GEO_COMP,NIVEL_TBL


In [21]:
sum_SEG_BT[sum_SEG_BT['CTMT'] == "PER763"]

,CTMT,TIP_INST_dic,FAS_CON,GEOM_CAB_dic,FORM_CAB_dic,BIT_FAS_1_dic,BIT_FAS_2_dic,BIT_FAS_3_dic,BIT_NEU_dic,MAT_FAS_1_dic,...,A2_NEU_dic,A3_NEU_dic,A4_NEU_dic,A5_NEU_dic,A6_NEU_dic,CND_FAS,QUANTIDADE,COMP,GEO_COMP,NIVEL_TBL


# SEGMENTOS  MT

In [22]:
SSDMT =  pd.read_pickle(path_info + 'SSDMT.pkl')
SSDMT.columns

Index(['COD_ID', 'PN_CON_1', 'PN_CON_2', 'CTMT', 'CT_COD_OP', 'UNI_TR_AT',
       'SUB', 'CONJ', 'ARE_LOC', 'DIST', 'PAC_1', 'PAC_2', 'FAS_CON',
       'TIP_INST', 'TIP_CND', 'POS', 'ODI', 'TI', 'CM', 'SITCONT', 'COMP',
       'DESCR', 'geometry', 'file', 'TIP_INST_dic', 'len'],
      dtype='object')

In [23]:
df_SSDMT = SSDMT[SSDMT['CTMT'].isin(alimentador['COD_ID'])]


### Distâncias (m) e Quantidades

In [24]:
check_MT = df_SSDMT.groupby(colunas_agrupadas[1]).agg({'DIST': 'count', 'COMP': 'sum','len' : 'sum' }).reset_index()

In [25]:
df_SSDMT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count', 'COMP': 'sum','len' : 'sum' }).reset_index()


,CTMT,TIP_INST_dic,FAS_CON,DIST,COMP,len


In [26]:

## Tucagem
# colunas_agrupadas_segcon = ['COD_ID','TUC_FAS', 'A1_FAS','A2_FAS','A3_FAS', 'A4_FAS', 'A5_FAS', 'A6_FAS']

## Personalizado
# colunas_agrupadas_segcon = ['COD_ID','GEOM_CAB','GEOM_CAB_dsc','FORM_CAB', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']

In [27]:
sum_SEG_MT = df_SSDMT.merge(SEGCON[colunas_agrupadas_segcon], left_on = 'TIP_CND', right_on = 'COD_ID', how ='left').groupby(colunas_agrupadas[1:] +colunas_agrupadas_segcon[1:] ).agg({'DIST': 'count', 'COMP': 'sum' ,'len' : 'sum' }).reset_index()
sum_SEG_MT.rename(columns={'DIST': 'QUANTIDADE', 'len': 'GEO_COMP'}, inplace=True)
sum_SEG_MT['NIVEL_TBL'] = 'MT'
sum_SEG_MT

,CTMT,TIP_INST_dic,FAS_CON,GEOM_CAB_dic,FORM_CAB_dic,BIT_FAS_1_dic,BIT_FAS_2_dic,BIT_FAS_3_dic,BIT_NEU_dic,MAT_FAS_1_dic,...,A2_NEU_dic,A3_NEU_dic,A4_NEU_dic,A5_NEU_dic,A6_NEU_dic,CND_FAS,QUANTIDADE,COMP,GEO_COMP,NIVEL_TBL


# Resultado

In [28]:
resultado_alimentadores = pd.concat([sum_SEG_BT,sum_SEG_MT], axis=0 )


In [29]:
resultado_alimentadores.columns

Index(['CTMT', 'TIP_INST_dic', 'FAS_CON', 'GEOM_CAB_dic', 'FORM_CAB_dic',
       'BIT_FAS_1_dic', 'BIT_FAS_2_dic', 'BIT_FAS_3_dic', 'BIT_NEU_dic',
       'MAT_FAS_1_dic', 'MAT_FAS_2_dic', 'MAT_FAS_3_dic', 'MAT_NEU_dic',
       'ISO_FAS_1_dic', 'ISO_FAS_2_dic', 'ISO_FAS_3_dic', 'ISO_NEU_dic',
       'TUC_FAS_dic', 'A1_FAS_dic', 'A2_FAS_dic', 'A3_FAS_dic', 'A4_FAS_dic',
       'A5_FAS_dic', 'A6_FAS_dic', 'TUC_NEU_dic', 'A1_NEU_dic', 'A2_NEU_dic',
       'A3_NEU_dic', 'A4_NEU_dic', 'A5_NEU_dic', 'A6_NEU_dic', 'CND_FAS',
       'QUANTIDADE', 'COMP', 'GEO_COMP', 'NIVEL_TBL'],
      dtype='object')

In [30]:
check_BT['NIVEL_TBL'] = "BT"
check_MT['NIVEL_TBL'] = "MT"

res_check = pd.concat([check_BT,check_MT], axis=0 )
res_check = resultado_alimentadores.groupby(['CTMT', 'NIVEL_TBL']).agg({'QUANTIDADE': 'sum', 'COMP': 'sum' ,'GEO_COMP' : 'sum' }).reset_index().merge(res_check, on =['CTMT','NIVEL_TBL'] )
res_check['DIF_quant'] = res_check['QUANTIDADE'] - res_check['DIST']
res_check['DIF_comp'] = res_check['COMP_x'] - res_check['COMP_y']


In [31]:
res_check[(res_check['DIF_quant'] !=- 0) | (res_check['DIF_comp'] != 0)]


,QUANTIDADE,COMP_x,GEO_COMP,CTMT,DIST,COMP_y,len,NIVEL_TBL,DIF_quant,DIF_comp


In [32]:
resultado_alimentadores.to_csv(path_res + 'res_alimentadores_BDGD.csv', sep=';', encoding='latin1', errors='ignore')


# Equipametos

### CRMT

In [131]:


un =  ["UNCRMT", "UN_CR"]
eq = ["EQCR"]

In [132]:
UNCRMT =   pd.read_pickle(path_info + un[0] + ".pkl")
UNCRMT.columns


Index(['COD_ID', 'DIST', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID', 'POT_NOM', 'PAC_1',
       'PAC_2', 'CTMT', 'UNI_TR_AT', 'SUB', 'CONJ', 'MUN', 'ARE_LOC',
       'DAT_CON', 'BANC', 'POS', 'DESCR', 'geometry', 'file', 'TIP_UNID_dic',
       'TIP_UNID_detail_dic', 'POT_NOM_UN_kVAr_dic'],
      dtype='object')

In [133]:
UNCRMT['CTMT']

0        PER763
1       VGE3919
2      CUR33000
3      GNB34723
4       VGE3758
         ...   
786    CMTTSL42
787     REC0001
788     CMR9663
789    CMTTSL42
790     CMR9663
Name: CTMT, Length: 791, dtype: object

In [134]:
df_UNCRMT = UNCRMT[UNCRMT['CTMT'].isin(alimentador['COD_ID'])]


In [135]:
# colunas_agrupadas = ['CTMT','DIST']
# colunas_agrupadas = ['DIST','CTMT', 'FAS_CON']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_UNID']
colunas_agrupadas = ['DIST','CTMT', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID_dic','TIP_UNID_detail_dic', 'POT_NOM_UN_kVAr_dic']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_CND','TIP_INST', 'FAS_CON']



In [136]:
df_UNCRMT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count' }).reset_index()

,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,POT_NOM_UN_kVAr_dic,DIST
0,PER763,ABC,AT,10_UNIDADE COMPENSADORA DE REATIVO,10_BANCO DE CAPACITORES PARALELO,3600,3
1,VRD014,ABC,AT,10_UNIDADE COMPENSADORA DE REATIVO,10_BANCO DE CAPACITORES PARALELO,600,1


In [137]:
EQCR =   pd.read_pickle(path_info + eq[0] + ".pkl")
EQCR.columns

Index(['COD_ID', 'DIST', 'TIP_INST', 'UN_CR', 'ODI', 'TI', 'CM', 'TUC', 'A1',
       'A2', 'A3', 'A4', 'A5', 'A6', 'UAR', 'IDUC', 'SITCONT', 'DAT_IMO',
       'DESCR', 'GRU_TEN', 'geometry', 'file', 'TUC_dic', 'TUC_A1',
       'TUC_A1_dic', 'TUC_A2', 'TUC_A2_dic', 'TUC_A3', 'TUC_A3_dic', 'TUC_A4',
       'TUC_A4_dic', 'TUC_A5', 'TUC_A5_dic', 'TUC_A6', 'TUC_A6_dic'],
      dtype='object')

In [138]:

## Tucagem
# colunas_agrupadas_eq = ['COD_ID', un[1],'TUC', 'A1','A2','A3', 'A4', 'A5', 'A6', 'GRU_TEN']

## Personalizado
colunas_agrupadas_eq = ['COD_ID','TIP_INST','UN_CR','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']

In [139]:
agg_dict = {col: lambda x: ', '.join(x.astype(str)) for col in colunas_agrupadas_eq[1:]}

# Realize o merge e, em seguida, agrupe e aplique as agregações
sum_EQCR_MT = df_UNCRMT.merge(EQCR[colunas_agrupadas_eq], left_on='COD_ID', right_on=un[1], how='left') \
    .groupby(colunas_agrupadas[1:]) \
    .agg({**agg_dict, 'DIST': 'count'}) \
    .reset_index()

sum_EQCR_MT['nivel_tbl'] = 'MT'
sum_EQCR_MT.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
# sum_EQCR_MT['Ativo'] = 'TRANSFORMADOR'
sum_EQCR_MT.head().style



,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,POT_NOM_UN_kVAr_dic,TIP_INST,UN_CR,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,nivel_tbl
0,PER763,ABC,AT,10_UNIDADE COMPENSADORA DE REATIVO,10_BANCO DE CAPACITORES PARALELO,3600,"SE_AT, SE_AT, SE_AT","1646507, 1646517, 1646512","125_BANCO DE CAPACITORES PARALELO, 125_BANCO DE CAPACITORES PARALELO, 125_BANCO DE CAPACITORES PARALELO","125.02_CLASSE DE TENSÃO INFERIOR A 69kV, 125.02_CLASSE DE TENSÃO INFERIOR A 69kV, 125.02_CLASSE DE TENSÃO INFERIOR A 69kV","125.15_anterior até 15,0kV, 125.15_anterior até 15,0kV, 125.15_anterior até 15,0kV","125.01_MANUAL, 125.01_MANUAL, 125.01_MANUAL","125.34_anterior até 3600 kVAR, 125.34_anterior até 3600 kVAR, 125.34_anterior até 3600 kVAR","125.00_N/A, 125.00_N/A, 125.00_N/A","125.00_N/A, 125.00_N/A, 125.00_N/A",3,MT
1,VRD014,ABC,AT,10_UNIDADE COMPENSADORA DE REATIVO,10_BANCO DE CAPACITORES PARALELO,600,SE_AT,18945375,125_BANCO DE CAPACITORES PARALELO,125.02_CLASSE DE TENSÃO INFERIOR A 69kV,"125.15_anterior até 15,0kV",125.01_MANUAL,125.19_anterior até 600 kVAR,125.00_N/A,125.00_N/A,1,MT


In [140]:
sum_EQCR_MT.to_csv(path_res + 'res_CRMT.csv', sep=';', encoding='latin1', errors='ignore')

### REMT

In [141]:
#REMT


un =  ["UNREMT", "UN_RE"]
eq = ["EQRE"]

In [142]:
UNREMT =   pd.read_pickle(path_info + un[0] + ".pkl")
UNREMT.columns


Index(['COD_ID', 'DIST', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID', 'TIP_REGU',
       'PAC_1', 'PAC_2', 'CTMT', 'UNI_TR_AT', 'SUB', 'CONJ', 'MUN', 'DAT_CON',
       'BANC', 'POS', 'DESCR', 'geometry', 'file', 'TIP_UNID_dic',
       'TIP_UNID_detail_dic'],
      dtype='object')

In [143]:
UNREMT['CTMT']

0       AGR008
1       TRS013
2       CEN003
3       CEN001
4       CAM020
        ...   
10     MRT2929
11      SCI006
12    SCO33618
13      STS006
14      CEN005
Name: CTMT, Length: 15, dtype: object

In [144]:
df_UNREMT = UNREMT[UNREMT['CTMT'].isin(alimentador['COD_ID'])]


In [145]:
# colunas_agrupadas = ['CTMT','DIST']
# colunas_agrupadas = ['DIST','CTMT', 'FAS_CON']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_UNID']
colunas_agrupadas = ['DIST','CTMT', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID_dic','TIP_UNID_detail_dic']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_CND','TIP_INST', 'FAS_CON']



In [146]:
df_UNREMT

,COD_ID,DIST,FAS_CON,SIT_ATIV,TIP_UNID,TIP_REGU,PAC_1,PAC_2,CTMT,UNI_TR_AT,...,CONJ,MUN,DAT_CON,BANC,POS,DESCR,geometry,file,TIP_UNID_dic,TIP_UNID_detail_dic
0,658201639,382,ABC,AT,14,DF,MT_639495,MT_639495_2,AGR008,1248369,...,15045,3304557,09/06/2020,1,PD,None,POINT (-43.29478 -22.81549),BD31122022_25022023/UNREMT.shp,14_UNIDADE REGULADORA,14_REGULADOR AUTOMÁTICO DE TENSÃO


In [147]:
df_UNREMT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count' }).reset_index()

,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,DIST
0,AGR008,ABC,AT,14_UNIDADE REGULADORA,14_REGULADOR AUTOMÁTICO DE TENSÃO,1


### EQRE

In [148]:
EQRE = pd.read_pickle(path_info + eq[0] + ".pkl")
EQRE.columns


Index(['COD_ID', 'DIST', 'TIP_INST', 'UN_RE', 'POT_NOM', 'TEN_REG',
       'LIG_FAS_P', 'LIG_FAS_S', 'COR_NOM', 'REL_TP', 'REL_TC', 'ODI', 'TI',
       'CM', 'TUC', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'UAR', 'IDUC',
       'SITCONT', 'DAT_IMO', 'PER_FER', 'PER_TOT', 'R', 'XHL', 'DESCR',
       'GRU_TEN', 'geometry', 'file', 'TUC_dic', 'TUC_A1', 'TUC_A1_dic',
       'TUC_A2', 'TUC_A2_dic', 'TUC_A3', 'TUC_A3_dic', 'TUC_A4', 'TUC_A4_dic',
       'TUC_A5', 'TUC_A5_dic', 'TUC_A6', 'TUC_A6_dic'],
      dtype='object')

In [149]:

## Tucagem
colunas_agrupadas_eq = ['COD_ID','TIP_INST','UN_RE','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']

## Personalizado
# colunas_agrupadas_eq = ['COD_ID','GEOM_CAB','GEOM_CAB_dsc','FORM_CAB', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']

In [150]:
colunas_agrupadas[1:]

['CTMT', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID_dic', 'TIP_UNID_detail_dic']

In [151]:
agg_dict = {col: lambda x: ', '.join(x.astype(str)) for col in colunas_agrupadas_eq[1:]}

# Realize o merge e, em seguida, agrupe e aplique as agregações
sum_EQRE_MT = df_UNREMT.merge(EQRE[colunas_agrupadas_eq], left_on='COD_ID', right_on=un[1], how='left') \
    .groupby(colunas_agrupadas[1:]) \
    .agg({**agg_dict, 'DIST': 'count'}) \
    .reset_index()

sum_EQRE_MT['nivel_tbl'] = 'MT'
sum_EQRE_MT.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
# sum_EQRE_MT['Ativo'] = 'TRANSFORMADOR'
sum_EQRE_MT.head().style

,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,TIP_INST,UN_RE,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,nivel_tbl
0,AGR008,ABC,AT,14_UNIDADE REGULADORA,14_REGULADOR AUTOMÁTICO DE TENSÃO,"RD_AER_URB, RD_AER_URB, RD_AER_URB","658201639, 658201639, 658201639","340_REGULADOR DE TENSÃO, 340_REGULADOR DE TENSÃO, 340_REGULADOR DE TENSÃO","340.01_REGULADOR DE TENSÃO, 340.01_REGULADOR DE TENSÃO, 340.01_REGULADOR DE TENSÃO","340.13_anterior até 13,8kV, 340.13_anterior até 13,8kV, 340.13_anterior até 13,8kV","340.54_anterior até 276 kVA, 340.54_anterior até 276 kVA, 340.54_anterior até 276 kVA","340.13_anterior até 200A, 340.13_anterior até 200A, 340.13_anterior até 200A","340.01_MONOFÁSICO / UNIPOLAR, 340.01_MONOFÁSICO / UNIPOLAR, 340.01_MONOFÁSICO / UNIPOLAR","340.00_N/A, 340.00_N/A, 340.00_N/A",3,MT


In [152]:
sum_EQRE_MT.to_csv(path_res + 'res_REMT.csv', sep=';', encoding='latin1', errors='ignore')

### SEMT

In [153]:
#seMT


un =  ["UNSEMT", "UN_SE"]
eq = ["EQSE"]

In [154]:
UNSEMT =   pd.read_pickle(path_info + un[0] + ".pkl")
UNSEMT.columns


Index(['COD_ID', 'DIST', 'PAC_1', 'PAC_2', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID',
       'P_N_OPE', 'CAP_ELO', 'COR_NOM', 'TLCD', 'DAT_CON', 'POS', 'CTMT',
       'UNI_TR_AT', 'SUB', 'CONJ', 'MUN', 'ARE_LOC', 'DESCR', 'geometry',
       'file', 'TIP_UNID_dic', 'TIP_UNID_detail_dic'],
      dtype='object')

In [155]:
df_UNSEMT = UNSEMT[UNSEMT['CTMT'].isin(alimentador['COD_ID'])]


In [156]:
# colunas_agrupadas = ['CTMT','DIST']
# colunas_agrupadas = ['DIST','CTMT', 'FAS_CON']
colunas_agrupadas = ['DIST','CTMT', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID_dic','TIP_UNID_detail_dic']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_REGU']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_CND','TIP_INST', 'FAS_CON']



In [157]:
df_UNSEMT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count' }).reset_index()

,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,DIST
0,AGR008,ABC,AT,19_UNIDADE SECCIONADORA,19_CHAVE FACA,19
1,AGR008,ABC,AT,22_UNIDADE SECCIONADORA,22_CHAVE FUSÍVEL,5
2,AGR008,ABC,AT,29_UNIDADE SECCIONADORA,29_DISJUNTOR,1
3,AGR008,ABC,AT,32_UNIDADE SECCIONADORA,32_RELIGADOR,1
4,AGR008,ABC,AT,35_UNIDADE SECCIONADORA,35_SECCIONALIZADOR,1
...,...,...,...,...,...,...
22,VRD014,ABC,AT,19_UNIDADE SECCIONADORA,19_CHAVE FACA,22
23,VRD014,ABC,AT,22_UNIDADE SECCIONADORA,22_CHAVE FUSÍVEL,9
24,VRD014,ABC,AT,29_UNIDADE SECCIONADORA,29_DISJUNTOR,1
25,VRD014,ABC,AT,32_UNIDADE SECCIONADORA,32_RELIGADOR,2


### EQSE

In [158]:
EQSE = pd.read_pickle(path_info + eq[0] + ".pkl")
EQSE.columns


Index(['COD_ID', 'DIST', 'TIP_INST', 'UN_SE', 'CLAS_TEN', 'ELO_FSV', 'MEI_ISO',
       'FAS_CON', 'COR_NOM', 'ODI', 'TI', 'CM', 'TUC', 'A1', 'A2', 'A3', 'A4',
       'A5', 'A6', 'UAR', 'IDUC', 'SITCONT', 'DAT_IMO', 'ABER_CRG', 'DESCR',
       'GRU_TEN', 'geometry', 'file', 'TUC_dic', 'TUC_A1', 'TUC_A1_dic',
       'TUC_A2', 'TUC_A2_dic', 'TUC_A3', 'TUC_A3_dic', 'TUC_A4', 'TUC_A4_dic',
       'TUC_A5', 'TUC_A5_dic', 'TUC_A6', 'TUC_A6_dic'],
      dtype='object')

In [159]:

## Tucagem
colunas_agrupadas_eq = ['COD_ID','TIP_INST','UN_SE','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']

## Personalizado
# colunas_agrupadas_eq = ['COD_ID','GEOM_CAB','GEOM_CAB_dsc','FORM_CAB', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']

In [160]:
colunas_agrupadas[1:]

['CTMT', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID_dic', 'TIP_UNID_detail_dic']

In [161]:
agg_dict = {col: lambda x: ', '.join(x.astype(str)) for col in colunas_agrupadas_eq[1:]}

# Realize o merge e, em seguida, agrupe e aplique as agregações
sum_EQSE_MT = df_UNSEMT.merge(EQSE[colunas_agrupadas_eq], left_on='COD_ID', right_on=un[1], how='left') \
    .groupby(colunas_agrupadas[1:]) \
    .agg({**agg_dict, 'DIST': 'count'}) \
    .reset_index()

sum_EQSE_MT['nivel_tbl'] = 'MT'
sum_EQSE_MT.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
# sum_EQSE_MT['Ativo'] = 'TRANSFORMADOR'
sum_EQSE_MT.head().style


,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,TIP_INST,UN_SE,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,nivel_tbl
0,AGR008,ABC,AT,19_UNIDADE SECCIONADORA,19_CHAVE FACA,"RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB","1601165, 1601165, 1601165, 1601354, 1601354, 1601354, 1601319, 1601319, 1601319, 1601214, 1601214, 1601214, 1601221, 1601221, 1601221, 1613625, 1613625, 1613625, 1601207, 1601207, 1601207, 1601179, 1601179, 1601179, 1601200, 1601200, 1601200, 1601172, 1601172, 1601172, 1601158, 1601158, 1601158, 1601312, 1601312, 1601312, 1601151, 1601151, 1601151, 1601340, 1601340, 1601340, 1601242, 1601242, 1601242, 1601368, 1601368, 1601368, 784002832, 784002832, 784002832, 1601263, 1601263, 1601263, 658201627, 658201627, 658201627","160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE, 160_CHAVE","160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA, 160.01_SECCIONADORA","160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterior até 15,0kV, 160.15_anterio

In [162]:
sum_EQSE_MT.to_csv(path_res + 'res_SEMT.csv', sep=';', encoding='latin1', errors='ignore')

### TRMT

In [163]:
#trMT


un =  ["UNTRMT", "UNI_TR_MT"]
eq = ["EQTRMT"]

In [164]:
UNTRMT =   pd.read_pickle(path_info + un[0] + ".pkl")
UNTRMT.columns


Index(['COD_ID', 'DIST', 'PAC_1', 'PAC_2', 'PAC_3', 'FAS_CON_P', 'FAS_CON_S',
       'FAS_CON_T', 'SIT_ATIV', 'TIP_UNID', 'POS', 'ATRB_PER', 'TEN_LIN_SE',
       'CAP_ELO', 'CAP_CHA', 'TAP', 'ARE_LOC', 'CONF', 'POSTO', 'POT_NOM',
       'PER_FER', 'PER_TOT', 'DAT_CON', 'CTMT', 'UNI_TR_AT', 'SUB', 'CONJ',
       'MUN', 'BANC', 'TIP_TRAFO', 'MRT', 'DESCR', 'geometry', 'file',
       'TIP_UNID_dic', 'TIP_UNID_detail_dic'],
      dtype='object')

In [165]:
df_UNTRMT = UNTRMT[UNTRMT['CTMT'].isin(alimentador['COD_ID'])]


In [166]:
# colunas_agrupadas = ['CTMT','DIST']
# colunas_agrupadas = ['DIST','CTMT', 'FAS_CON']
colunas_agrupadas = ['DIST','CTMT', 'SIT_ATIV','POT_NOM', 'TIP_UNID_dic','TIP_UNID_detail_dic']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_REGU']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_CND','TIP_INST', 'FAS_CON']



In [167]:
colunas_agrupadas[1:]

['CTMT', 'SIT_ATIV', 'POT_NOM', 'TIP_UNID_dic', 'TIP_UNID_detail_dic']

In [168]:
df_UNTRMT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count' }).reset_index()

,CTMT,SIT_ATIV,POT_NOM,TIP_UNID_dic,TIP_UNID_detail_dic,DIST
0,AGR008,AT,45.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,2
1,AGR008,AT,75.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,6
2,AGR008,AT,112.5,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,15
3,AGR008,AT,150.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,12
4,AGR008,AT,225.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,13
...,...,...,...,...,...,...
36,VRD014,AT,112.5,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,10
37,VRD014,AT,150.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,8
38,VRD014,AT,225.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,2
39,VRD014,AT,300.0,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENS...,3


### EQTR

In [169]:
EQTRMT = pd.read_pickle(path_info + eq[0] + ".pkl")
EQTRMT.columns


Index(['COD_ID', 'DIST', 'TIP_INST', 'UNI_TR_MT', 'CLAS_TEN', 'POT_NOM', 'LIG',
       'FAS_CON', 'TEN_PRI', 'TEN_SEC', 'TEN_TER', 'LIG_FAS_P', 'LIG_FAS_S',
       'LIG_FAS_T', 'ODI', 'TI', 'CM', 'TUC', 'A1', 'A2', 'A3', 'A4', 'A5',
       'A6', 'UAR', 'IDUC', 'SITCONT', 'DAT_IMO', 'PER_FER', 'PER_TOT', 'R',
       'XHL', 'XHT', 'XLT', 'DESCR', 'geometry', 'file', 'POT_NOM_dic',
       'TEN_PRI_dic', 'TEN_SEC_dic', 'TEN_TER_dic', 'TUC_dic', 'TUC_A1',
       'TUC_A1_dic', 'TUC_A2', 'TUC_A2_dic', 'TUC_A3', 'TUC_A3_dic', 'TUC_A4',
       'TUC_A4_dic', 'TUC_A5', 'TUC_A5_dic', 'TUC_A6', 'TUC_A6_dic'],
      dtype='object')

In [170]:

## Tucagem
colunas_agrupadas_eq = ['COD_ID','TIP_INST','POT_NOM_dic','UNI_TR_MT','TEN_PRI_dic','TEN_SEC_dic','TEN_TER_dic','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']

## Personalizado
# colunas_agrupadas_eq = ['COD_ID','GEOM_CAB','GEOM_CAB_dsc','FORM_CAB', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']

In [171]:
colunas_agrupadas[1:]

['CTMT', 'SIT_ATIV', 'POT_NOM', 'TIP_UNID_dic', 'TIP_UNID_detail_dic']

In [172]:
agg_dict = {col: lambda x: ', '.join(x.astype(str)) for col in colunas_agrupadas_eq[1:]}

# Realize o merge e, em seguida, agrupe e aplique as agregações
sum_EQTRMT_MT = df_UNTRMT.merge(EQTRMT[colunas_agrupadas_eq], left_on='COD_ID', right_on=un[1], how='left') \
    .groupby(colunas_agrupadas[1:]) \
    .agg({**agg_dict, 'DIST': 'count'}) \
    .reset_index()

sum_EQTRMT_MT['nivel_tbl'] = 'MT'
sum_EQTRMT_MT.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
# sum_EQTRMT_MT['Ativo'] = 'TRANSFORMADOR'
sum_EQTRMT_MT.head().style

,CTMT,SIT_ATIV,POT_NOM,TIP_UNID_dic,TIP_UNID_detail_dic,TIP_INST,POT_NOM_dic,UNI_TR_MT,TEN_PRI_dic,TEN_SEC_dic,TEN_TER_dic,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,nivel_tbl
0,AGR008,AT,45.000000,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENSÃO (MT/BT),"RD_AER_URB, RD_AER_URB","45, 45","801031, 800959","49_13800, 49_13800","10_220, 10_220","0_0, 0_0","565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO","565.01_AÉREO, 565.01_AÉREO","565.13_anterior até 13,8kV, 565.13_anterior até 13,8kV","565.91_Anterior até 550 kV, 565.91_Anterior até 550 kV","565.23_anterior até 45 kVA, 565.23_anterior até 45 kVA","565.03_TRIFÁSICO / TRIPOLAR, 565.03_TRIFÁSICO / TRIPOLAR","565.01_MONOFÁSICO / UNIPOLAR, 565.01_MONOFÁSICO / UNIPOLAR",2,MT
1,AGR008,AT,75.000000,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENSÃO (MT/BT),"RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB","75, 75, 75, 75, 75, 75","801283, 800833, 800965, 800347, 868666, 868552","46_13200, 46_13200, 49_13800, 49_13800, 49_13800, 49_13800","10_220, 10_220, 10_220, 10_220, 10_220, 10_220","0_0, 0_0, 0_0, 0_0, 0_0, 0_0","565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO","565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO","565.13_anterior até 13,8kV, 565.13_anterior até 13,8kV, 565.13_anterior até 13,8kV, 565.13_anterior até 13,8kV, 565.13_anterior até 13,8kV, 565.13_anterior até 13,8kV","565.91_Anterior até 550 kV, 565.91_Anterior até 550 kV, 565.91_Anterior até 550 kV, 565.91_Anterior até 550 kV, 565.91_Anterior até 550 kV, 565.91_Anterior até 550 kV","565.29_anterior até 75 kVA, 565.29_anterior até 75 kVA, 565.29_anterior até 75 kVA, 565.29_anterior até 75 kVA, 565.29_anterior até 75 kVA, 565.29_anterior até 75 kVA","565.03_TRIFÁSICO / TRIPOLAR, 565.03_TRIFÁSICO / TRIPOLAR, 565.03_TRIFÁSICO / TRIPOLAR, 565.03_TRIFÁSICO / TRIPOLAR, 565.03_TRIFÁSICO / TRIPOLAR, 565.03_TRIFÁSICO / TRIPOLAR","565.02_BIFÁSICO, 565.01_MONOFÁSICO / UNIPOLAR, 565.01_MONOFÁSICO / UNIPOLAR, 565.02_BIFÁSICO, 565.02_BIFÁSICO, 565.01_MONOFÁSICO / UNIPOLAR",6,MT
2,AGR008,AT,112.500000,38_UNIDADE TRANSFORMADORA,38_TRANSFORMADOR DE DISTRIBUIÇÃO DE MÉDIA TENSÃO (MT/BT),"RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB, RD_AER_URB","112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5, 112.5","801019, 801025, 800011, 799921, 800923, 800329, 793215, 800677, 870994, 34313369, 98886555, 70855355, 298355709, 457435547, 512712204","49_13800, 49_13800, 46_13200, 49_13800, 49_13800, 49_13800, 46_13200, 46_13200, 46_13200, 46_13200, 46_13200, 46_13200, 46_13200, 49_13800, 46_13200","10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220, 10_220","0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0, 0_0","565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO, 565_TRANSFORMADOR DE DISTRIBUIÇÃO","565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO, 565.01_AÉREO","565.13_anterior até 13,8kV, 565.13_anterior até 

In [173]:
sum_EQTRMT_MT.to_csv(path_res + 'res_TRMT.csv', sep=';', encoding='latin1', errors='ignore')

### SEBT

In [174]:
#trMT


un =  ["UNSEBT", "UN_SE"]
eq = ["EQSE"]

In [175]:
UNSEBT =   pd.read_pickle(path_info + un[0] + ".pkl")
UNSEBT.columns


Index(['COD_ID', 'DIST', 'PAC_1', 'PAC_2', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID',
       'P_N_OPE', 'CAP_ELO', 'COR_NOM', 'TLCD', 'DAT_CON', 'POS', 'UNI_TR_MT',
       'CTMT', 'UNI_TR_AT', 'SUB', 'CONJ', 'MUN', 'ARE_LOC', 'DESCR',
       'geometry', 'file', 'TIP_UNID_dic', 'TIP_UNID_detail_dic'],
      dtype='object')

In [176]:
df_UNSEBT = UNSEBT[UNSEBT['CTMT'].isin(alimentador['COD_ID'])]


In [177]:
# colunas_agrupadas = ['CTMT','DIST']
# colunas_agrupadas = ['DIST','CTMT', 'FAS_CON']
colunas_agrupadas = ['DIST','CTMT', 'FAS_CON', 'SIT_ATIV','TIP_UNID_dic', 'TIP_UNID_detail_dic', ]
# colunas_agrupadas = ['DIST','CTMT', 'TIP_REGU']
# colunas_agrupadas = ['DIST','CTMT', 'TIP_CND','TIP_INST', 'FAS_CON']



In [178]:
colunas_agrupadas[1:]

['CTMT', 'FAS_CON', 'SIT_ATIV', 'TIP_UNID_dic', 'TIP_UNID_detail_dic']

In [179]:
df_UNSEBT.groupby(colunas_agrupadas[1:]).agg({'DIST': 'count' }).reset_index()

,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,DIST
0,TQR60111,ABCN,AT,46_UNIDADE SECCIONADORA,46_FUSÍVEL,39
1,VRD014,ABCN,AT,46_UNIDADE SECCIONADORA,46_FUSÍVEL,4


### EQSE

In [180]:
EQSE.columns


Index(['COD_ID', 'DIST', 'TIP_INST', 'UN_SE', 'CLAS_TEN', 'ELO_FSV', 'MEI_ISO',
       'FAS_CON', 'COR_NOM', 'ODI', 'TI', 'CM', 'TUC', 'A1', 'A2', 'A3', 'A4',
       'A5', 'A6', 'UAR', 'IDUC', 'SITCONT', 'DAT_IMO', 'ABER_CRG', 'DESCR',
       'GRU_TEN', 'geometry', 'file', 'TUC_dic', 'TUC_A1', 'TUC_A1_dic',
       'TUC_A2', 'TUC_A2_dic', 'TUC_A3', 'TUC_A3_dic', 'TUC_A4', 'TUC_A4_dic',
       'TUC_A5', 'TUC_A5_dic', 'TUC_A6', 'TUC_A6_dic'],
      dtype='object')

In [181]:

## Tucagem
colunas_agrupadas_eq = ['COD_ID','TIP_INST','UN_SE','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']


## Personalizado
# colunas_agrupadas_eq = ['COD_ID','GEOM_CAB','GEOM_CAB_dsc','FORM_CAB', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']

In [182]:
agg_dict = {col: lambda x: ', '.join(x.astype(str)) for col in colunas_agrupadas_eq[1:]}

# Realize o merge e, em seguida, agrupe e aplique as agregações
sum_EQSE_BT = df_UNSEBT.merge(EQSE[colunas_agrupadas_eq], left_on='COD_ID', right_on=un[1], how='left') \
    .groupby(colunas_agrupadas[1:]) \
    .agg({**agg_dict, 'DIST': 'count'}) \
    .reset_index()

sum_EQSE_BT['nivel_tbl'] = 'BT'
sum_EQSE_BT.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
# sum_EQSE_BT['Ativo'] = 'TRANSFORMADOR'
sum_EQSE_BT.head().style

,CTMT,FAS_CON,SIT_ATIV,TIP_UNID_dic,TIP_UNID_detail_dic,TIP_INST,UN_SE,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,nivel_tbl
0,TQR60111,ABCN,AT,46_UNIDADE SECCIONADORA,46_FUSÍVEL,"RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB, RD_SUBT_URB","322067249, 322067256, 328129283, 328129290, 328129297, 328129276, 327659748, 327659755, 327659762, 327659769, 327659776, 327542262, 327542269, 327181970, 327181977, 327181984, 327181991, 327542276, 327542283, 327542290, 600948317, 600948331, 600948345, 600948353, 600948361, 600948572, 600948586, 600948594, 600948602, 328425094, 328425108, 328425101, 328425115, 328425122, 328425157, 328425129, 328425136, 328425143, 328425150","nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan","000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO, 000.00_NAO INFO","000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A","000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A","000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A","000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A","000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/A, 000.00_N/

In [183]:
sum_EQSE_BT.to_csv(path_res + 'res_SEBT.csv', sep=';', encoding='latin1', errors='ignore')

### PONNOT

In [184]:
df_SSDBT_PN_1 = df_SSDBT[['CTMT', 'PN_CON_1']].rename(columns={'PN_CON_1': 'PN'})
df_SSDBT_PN_2 = df_SSDBT[['CTMT', 'PN_CON_2']].rename(columns={'PN_CON_2': 'PN'})
df_SSDMT_PN_1 = df_SSDMT[['CTMT', 'PN_CON_1']].rename(columns={'PN_CON_1': 'PN'})
df_SSDMT_PN_2 = df_SSDMT[['CTMT', 'PN_CON_2']].rename(columns={'PN_CON_2': 'PN'})
alimentador_pn = pd.concat([df_SSDBT_PN_1, df_SSDBT_PN_2,df_SSDMT_PN_1,df_SSDMT_PN_2]).drop_duplicates()
alimentador_pn


,CTMT,PN
230023,VRD014,55677930
230307,VRD014,55677937
230308,VRD014,55677944
230311,VRD014,240064824
231809,VRD014,277469217
...,...,...
889119,TQR60111,15698921
900860,SRP009,655290215
922602,VRD014,251122478
926348,VRD014,300043327


In [4]:
PONNOT = pd.read_pickle(path_info + 'PONNOT.pkl')
PONNOT.columns

Index(['COD_ID', 'DIST', 'TIP_PN', 'TIP_INST', 'POS', 'ESTR', 'MAT', 'ESF',
       'ALT', 'ARE_LOC', 'CONJ', 'MUN', 'ODI', 'TI', 'CM', 'TUC', 'A1', 'A2',
       'A3', 'A4', 'A5', 'A6', 'UAR', 'SITCONT', 'DESCR', 'geometry', 'file',
       'TIP_PN_dic', 'ESTR_dic', 'MAT_dic', 'TUC_dic', 'TUC_A1', 'TUC_A1_dic',
       'TUC_A1_A2', 'TUC_A2_dic', 'TUC_A1_A3', 'TUC_A3_dic', 'TUC_A1_A4',
       'TUC_A4_dic', 'TUC_A1_A5', 'TUC_A5_dic', 'TUC_A1_A6', 'TUC_A6_dic'],
      dtype='object')

In [186]:

## Tucagem
colunas_agrupadas_pn = ['COD_ID','TIP_INST','TIP_PN_dic','ESTR_dic','MAT_dic','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']
colunas_agrupadas = ['CTMT']
## Personalizado
# colunas_agrupadas_pn = ['COD_ID','GEOM_CAB','GEOM_CAB_dsc','FORM_CAB', 'MAT_FAS_1','TUC_FAS', 'A1_FAS','A2_FAS']

In [187]:
(colunas_agrupadas_pn[1:])

['TIP_INST',
 'TIP_PN_dic',
 'ESTR_dic',
 'MAT_dic',
 'TUC_dic',
 'TUC_A1_dic',
 'TUC_A2_dic',
 'TUC_A3_dic',
 'TUC_A4_dic',
 'TUC_A5_dic',
 'TUC_A6_dic']

In [188]:
# agg_dict = {col: lambda x: ', '.join(x.astype(str)) for col in colunas_agrupadas_pn}

# # Realize o merge e, em seguida, agrupe e aplique as agregações
# sum_PN = alimentador_pn.merge(PONNOT, left_on='PN', right_on='COD_ID', how='left') \
#     .groupby(colunas_agrupadas) \
#     .agg({**agg_dict, 'DIST': 'count'}) \
#     .reset_index()

# # sum_EQSE_BT['nivel_tbl'] = 'BT'
# sum_PN.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
# # sum_EQSE_BT['Ativo'] = 'TRANSFORMADOR'
# sum_PN.head().style

In [189]:
colunas_agrupadas_pn = ['CTMT','TIP_INST','TIP_PN_dic','ESTR_dic','MAT_dic','TUC_dic', 'TUC_A1_dic','TUC_A2_dic','TUC_A3_dic', 'TUC_A4_dic', 'TUC_A5_dic', 'TUC_A6_dic']
colunas_agrupadas = ['CTMT']

In [190]:
sum_PN = alimentador_pn.merge(PONNOT, left_on = 'PN', right_on = 'COD_ID', how ='left').groupby(colunas_agrupadas_pn ).agg({'DIST': 'count'}).reset_index()#'len' : 'sum' }).reset_index()
sum_PN['Ativo'] = 'POSTE'
sum_PN.rename(columns={'DIST': 'QUANTIDADE'}, inplace=True)
sum_PN

,CTMT,TIP_INST,TIP_PN_dic,ESTR_dic,MAT_dic,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,Ativo
0,AGR008,LD_AER_AT,Poste,Duplo T,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.02_DUPLO T,255.01.01_CONCRETO,"255.01.10_10 m; 10,5 m",255.01.35_anterior até 1000 daN,255PST.00_N/A,1,POSTE
1,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.07_7 m; 7,5 m",255.01.10_anterior até 100 daN,255PST.00_N/A,2,POSTE
2,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.07_7 m; 7,5 m",255.01.16_anterior até 300 daN,255PST.00_N/A,4,POSTE
3,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.08_8 m; 8,5 m",255.01.16_anterior até 300 daN,255PST.00_N/A,2,POSTE
4,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.09_9 m,255.01.10_anterior até 100 daN,255PST.00_N/A,1,POSTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.12_12 m,255.01.35_anterior até 1000 daN,255PST.00_N/A,7,POSTE
118,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.18_anterior até 400 daN,255PST.00_N/A,8,POSTE
119,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.20_anterior até 600 daN,255PST.00_N/A,3,POSTE
120,VRD014,RD_AER_URB,Poste,Circular,Madeira,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.02_MADEIRA,255.01.11_11 m,255.01.16_anterior até 300 daN,255PST.00_N/A,3,POSTE


In [191]:
sum_PN.to_csv(path_res + 'res_PN.csv', sep=';', encoding='latin1', errors='ignore')

# Exportacao

In [196]:


# Iterar Sobre os circuitos
for ali in alimentador['COD_ID']:
    # Crie um novo arquivo Excel para cada circuito
    output_file = path_excel + ali + '.xlsx'
    workbook = xlsxwriter.Workbook(output_file)

    # Iterar sobre os DataFrames e salvar cada um em uma aba
    dataframes = {
        'SEGBT': sum_SEG_BT,
        'SEGMT': sum_SEG_MT,
        'CRMT': sum_EQCR_MT,
        'REMT': sum_EQRE_MT,
        'SEMT': sum_EQSE_MT,
        'TRMT': sum_EQTRMT_MT,
        'SEBT': sum_EQSE_BT,
        'PONNOT': sum_PN
    }

    for sheet_name, df in dataframes.items():
        # Crie uma nova aba no arquivo Excel
        worksheet = workbook.add_worksheet(sheet_name)

        df_sub = df[df['CTMT'] == ali]

        # Salve o DataFrame na aba
        for col_num, value in enumerate(df_sub.columns):
            worksheet.write(0, col_num, value)  # Escreva os cabeçalhos das colunas

        for row, data in enumerate(df_sub.values):
            worksheet.write_row(row + 1, 0, data)  # Comece a escrever os dados na segunda linha

    # Feche o arquivo Excel
    workbook.close()


In [197]:
df_sub

,CTMT,TIP_INST,TIP_PN_dic,ESTR_dic,MAT_dic,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,Ativo
102,VRD014,LD_AER_AT,Poste,Duplo T,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.02_DUPLO T,255.01.01_CONCRETO,"255.01.10_10 m; 10,5 m",255.01.35_anterior até 1000 daN,255PST.00_N/A,1,POSTE
103,VRD014,RD_AER_RUR,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.26_anterior até 800 daN,255PST.00_N/A,1,POSTE
104,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.09_9 m,255.01.16_anterior até 300 daN,255PST.00_N/A,27,POSTE
105,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.09_9 m,255.01.18_anterior até 400 daN,255PST.00_N/A,2,POSTE
106,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.10_10 m; 10,5 m",255.01.14_anterior até 200 daN,255PST.00_N/A,13,POSTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.12_12 m,255.01.35_anterior até 1000 daN,255PST.00_N/A,7,POSTE
118,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.18_anterior até 400 daN,255PST.00_N/A,8,POSTE
119,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.20_anterior até 600 daN,255PST.00_N/A,3,POSTE
120,VRD014,RD_AER_URB,Poste,Circular,Madeira,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.02_MADEIRA,255.01.11_11 m,255.01.16_anterior até 300 daN,255PST.00_N/A,3,POSTE


In [198]:
df

,CTMT,TIP_INST,TIP_PN_dic,ESTR_dic,MAT_dic,TUC_dic,TUC_A1_dic,TUC_A2_dic,TUC_A3_dic,TUC_A4_dic,TUC_A5_dic,TUC_A6_dic,QUANTIDADE,Ativo
0,AGR008,LD_AER_AT,Poste,Duplo T,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.02_DUPLO T,255.01.01_CONCRETO,"255.01.10_10 m; 10,5 m",255.01.35_anterior até 1000 daN,255PST.00_N/A,1,POSTE
1,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.07_7 m; 7,5 m",255.01.10_anterior até 100 daN,255PST.00_N/A,2,POSTE
2,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.07_7 m; 7,5 m",255.01.16_anterior até 300 daN,255PST.00_N/A,4,POSTE
3,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,"255.01.08_8 m; 8,5 m",255.01.16_anterior até 300 daN,255PST.00_N/A,2,POSTE
4,AGR008,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.09_9 m,255.01.10_anterior até 100 daN,255PST.00_N/A,1,POSTE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.12_12 m,255.01.35_anterior até 1000 daN,255PST.00_N/A,7,POSTE
118,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.18_anterior até 400 daN,255PST.00_N/A,8,POSTE
119,VRD014,RD_AER_URB,Poste,Circular,Concreto,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.01_CONCRETO,255.01.14_14 m,255.01.20_anterior até 600 daN,255PST.00_N/A,3,POSTE
120,VRD014,RD_AER_URB,Poste,Circular,Madeira,"255_ESTRUTURA (POSTE, TORRE)",255.01_POSTE,255.01.01_CIRCULAR,255.01.02_MADEIRA,255.01.11_11 m,255.01.16_anterior até 300 daN,255PST.00_N/A,3,POSTE


In [199]:
ali

'VRD014'